# A journey through Sweden!
Hej, välkommen till Sverige! 
In this Notebook we will go on a journey through Sweden while analyzing and manipulating a lot of different types of signals. All the different types of signals play an important role in Swedens culture and identity of the country - So buckle up for an interesting ride!

We will start on the northernmost part of Sweden and go all the way through to the south.

#### Packing up our backpack
Before we explore Swedens vast cultural and natural vastness, we need a couple of tools and notes on where we can find our artifacts.

In [ ]:
import numpy as np
from pandas import read_csv
import cv2
from matplotlib import pyplot as plt
import librosa
from scipy.signal import stft, istft
from IPython.display import Audio, display
import time

IMAGE_FOLDER = 'images/'
AUDIO_FOLDER = 'audio/'
DATA_FOLDER = 'data/'

# The first stop | Task 1.1 - Image Properties
We start our journey in [Kiruna](https://google.com/maps/place/Kiruna,+Sweden), a remote little town in Swedens "Norrland". Many tourists like to visit this place in the cold winters in order to see the unique **Northern Lights**... And good news, so will we!

In this first task we will demonstrate following properties using a picture of a northern light:
- Sharpness
- Noise
- Different color spaces (correction or transformations)

Sadly, I didn't have a state of the art cam with me when shooting that picture, so we will have to try our best to enhance its quality by using the tools we brought with us.

## Sharpness
Sharpness in images refers to the clarity of detail and the distinctness of structures or boundaries within the image. It affects the contrast at edges and fine details, making them either clear and well-defined or blurry and indistinct. An image is perceived as sharp when edges and textures are distinct, whereas a loss of sharpness, or blurriness, causes these features to merge and become less distinguishable.

#### Sharpening Image using Unsharp masking technique
The [Unsharp Masking](https://www.wikiwand.com/en/Unsharp_masking) technique is a commonly used method of applying sharpness to an image. It works like this:
1. Start by creating a blurred version of the original image, typically using a Gaussian 
2. Subtract the blurred image from the original. This gives the detail or "unsharp mask" of the image, highlighting the areas of contrast
3. Add the unsharp mask back to the original image. This accentuates the edges and increases contrast, enhancing perceived sharpness

Unsharp masking sharpens an image by emphasizing the difference between the original image and its blurred version.

In [ ]:
def sharpen_image(image, scaling_factor=0.2):
    image = image.astype(np.float32) / 255.0

    blurred_image = cv2.GaussianBlur(image, (0, 0), sigmaX=2)

    mask = image - blurred_image

    sharpened_image = image + scaling_factor * mask

    sharpened_image = np.clip(sharpened_image, 0, 1).astype(np.float32)
    
    return sharpened_image

#### Measuring Sharpness with Eigenvalues
The Paper [Image sharpness measure using eigenvalues](https://ieeexplore.ieee.org/abstract/document/4697259) states that it is possible to use the first 6 Eigenvalues (traced in $SVD$) to measure the sharpness of an image. To express this theory mathematically, a 6-Step procedure was implemented in the following function:
1. Convert the image to greyscale
2. Compute the `mean_operator` $\mu=\frac{1}{N\times M}\cdot I$, where $N$ and $M$ are the dimensions in each axis
3. Calculate the mean deviation matrix $G=I-\mu$
4. Compute the Covariance Matrix $S=G\times G^T \cdot\frac{1}{N-1}$ - This matrix captures the pixels relationships when looking at intensities
5. Singluar Value Decomposition: Apply SVD (`np.linalg.svd()`) on the Covariance Matrix $S$ to get $U,\sum,V$ where $\sum$ is the diagonal matrix of eigenvalues
6. Summing the first 6 eigenvalues by `np.sum(eigenvalues[:6])` or mathematically $trace(S_6)=\sum_{i=1}^6 s_i$ where $S$ would now be the matrix of the square root eigenvalues and $s$ the diagonal elements

In [ ]:
def get_eigenvals_sharpness(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    mean_operator = image / (image.shape[0] * image.shape[1])

    G = image - mean_operator

    S = (G@G.T)/(image.shape[0]-1)

    U, s, V = np.linalg.svd(S)

    eigenvalues = s[::-1]**2

    return np.sum(eigenvalues[:6])

#### Measuring Sharpness with Laplacian (2nd order derivative)
With this metric, the sharpness of an image can be measured by the Variance of the Laplaian. The Laplacian is a 2D isotropic measure of the second spatial derivative of an image. In simple terms, it measures the rate of change or variations in the image. The sharper the image, the higher the rate of change will be in areas with edges or fine details, resulting in higher values in the Laplacian image.

Analytically, the Laplacian for a 2-dimensional image $f(x,y)$ is defined as: $$\nabla^2f=\frac{\partial^2 f}{\partial x^2}+\frac{\partial^2 f}{\partial y^2}$$

In discrete terms, to get the laplacian of an image, usually is derived from the appliance of a Laplacian kernel which looks like $$\begin{bmatrix}0 & -1 & 0 \\ -1 & 5 & -1 \\ 0 & -1 & 0 \end{bmatrix}$$
In a later chapter we are also going to implement such a filter but for mathematical simplicity's sake I didn't implement the Laplacian using the classical approach to derivatives.

In [ ]:
def get_laplacian_sharpness(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    laplacian = cv2.Laplacian(image, cv2.CV_32F)
    return laplacian.var()

#### Measuring Sharpness with L2 of gradient of image
A similar approach to measuring sharpness with the Laplacian woul be to measure it using the $L_2$ of the $\nabla f$ gradient. I am not sure if this metric actually exists but it made sense to me so I started to experiment with it and it turned out to be very similar to the Laplacian and it seemed to capture the change in sharpness.

The thought that led me to this approach was that in theory, sharpness characterizes itself as abrupt changes in color/contrast which would result in a high slope-value if differentiated. So taking the first derivative instead of the second (Laplacian) and then applying the $L_2$-Norm would describe the combination of the derivative in $x$ and $y$ direction sufficiently.

In [ ]:
def get_gradient_sharpness(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    gy, gx = np.gradient(image)
    gnorm = np.sqrt(gx**2 + gy**2)
    
    return np.average(gnorm)

#### Defining experiments of Sharpness
For each experiment I define a function that will give us resulting information. In this experiment, we run through `n_steps` (which will default to 4). In each step we apply more sharpness and then calculate the applied sharpness using the three metrics we just went through before this step. After all the steps have gone through, we will display the calculated metrics at each step so we can make out how robust these metrics actually are.

In [ ]:
def experiment_sharpness(image, sharpness_factor=0.2, n_steps=4):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')
    plt.show()

    processed_images = []
    metrics = {
        'eigenvals': [],
        'laplacian': [],
        'gradient': []
    }

    titles = ['Eigenvalue Sharpness Value', 'Laplacian Sharpness Value', 'Gradient Sharpness Value']
    ylabels = ['Eigenvalue Sharpness', 'Laplacian Sharpness', 'Gradient Sharpness']

    for i in range(n_steps):
        print(f'Applying Sharpness at step {i+1}/{n_steps}')
        sharpened_image = sharpen_image(image, i*sharpness_factor)
        processed_images.append(sharpened_image)
        eigenvals_sharpness = get_eigenvals_sharpness(sharpened_image)
        laplacian_sharpness = get_laplacian_sharpness(sharpened_image)
        gradient_sharpness = get_gradient_sharpness(sharpened_image)
        
        metrics['eigenvals'].append(eigenvals_sharpness)
        metrics['laplacian'].append(laplacian_sharpness)
        metrics['gradient'].append(gradient_sharpness)

    images_fig, ax = plt.subplots(1, n_steps, figsize=(20, 4))
    images_fig.suptitle('Images at each Sharpening Step', fontsize=16)

    for i in range(n_steps):
        rgb_image = cv2.cvtColor(processed_images[i], cv2.COLOR_BGR2RGB) 
        ax[i].imshow(rgb_image, cmap='gray')
        ax[i].axis('off')
        ax[i].set_title(f'Step {i+1}', fontsize=12)

    plt.show()

    metrics_fig, ax = plt.subplots(1, 3, figsize=(20, 5))
    metrics_fig.suptitle('Sharpness Metrics', fontsize=16)
    
    for i, (key, values) in enumerate(metrics.items()):
        ax[i].plot(range(n_steps), values, label=f'{key.capitalize()} Sharpness', color='b')
        ax[i].set_xlabel('Step')
        ax[i].set_title(titles[i], fontsize=12)
        ax[i].set_ylabel(ylabels[i])
        ax[i].set_xticks(range(n_steps))
        ax[i].grid('on')

    plt.show()

##### Experiment 1 - Northern Lights
The goal of this experiment is to apply sharpness to a rather blurry and smoothed image of the northern lights so that the outlines become clearer.

In [ ]:
northern_lights = cv2.imread(IMAGE_FOLDER+'northern_lights.png', cv2.IMREAD_COLOR)

experiment_sharpness(northern_lights, sharpness_factor=8, n_steps=4)

The metrics all show a steady growth. The Gradient Sharpness growth looks linear while the Laplacian and Eigenvalue Sharpness Metrics both show a rather exponential curve. All metrics capture a positive change of sharpness. Regarding the image itself, the image seems to have increased in sharpness - Mostly noticable in the bottom right tree group.

##### Experiment 2
This [Stone Circle in Kiruna](https://maps.app.goo.gl/MzPD3arxS5YK3xaE9) was built by Vikings in the Iron Age and used for ceremony gatherings, burials, and offerings to the norse gods.
In this experiment we expect to see a drastic change in the snowy ground - The upper half of the image already has a lot of structure while the ground is merely just a white area. Through sharpening we should get more information on the ground's toplogical structure.

In [ ]:
stone_circle = cv2.imread(IMAGE_FOLDER+'stone_circle.png', cv2.IMREAD_COLOR)

experiment_sharpness(stone_circle, sharpness_factor=8, n_steps=4)

Both the Laplacian and Gradient Sharpness Metric seem to follow a converging pattern while the Eigenvalue Sharpness seems to steadily grow throughout applying sharpness. Though we cannot really say if the latter metrics actually converge as we only go through `4` steps here. They all seem to capture a positive change in sharpness, which means they also stay stable in an image with more structure. The ground looks more detailed and the toplogy can definitely be recognized.

##### Experiment 3
Before we look at noise metrics, lets look at how our proposed sharpness metrics perform when we sharpen a noisy image. This image was taken in the [Stora Sjöfallets Nationalpark](https://maps.app.goo.gl/Xyxyab1Y6pZVYE7h8), a vast natural habitat. This picture displays the Akkajaura Lake and it seems to have a lot of noise on it - Therefore the goal of this experiment is to test stability of the metrics if the image is noisy.

In [ ]:
sjofallets = cv2.imread(IMAGE_FOLDER+'nationalpark.jpg', cv2.IMREAD_COLOR)

experiment_sharpness(sjofallets, sharpness_factor=4, n_steps=8)

In this experiment we went through 8 steps of image alteration. Interestingly we can now definietly see a converging pattern emerging from our metrics in all three metrics the more sharpness we add to the noisy image. Both the Laplacian and Gradient Sharpness roughly look the same while the Eigenvalue Sharpness curve seems to have a more unique structure, especially at step `3` where it "loses" sharpness until step `5`.

## Noise
Noise in images refers to random variations in brightness or color information, often resembling grain or specks. It arises from sources like electronic interference, poor sensor quality, or inadequate lighting conditions. Visually, noise can manifest as random dots, splotches, or grainy patterns, diminishing an image's clarity and sharpness.

To look at noise during multiple transformations, we define a **denoising algorithm** to edit images that we then can measure noise on. For that I defined following function that makes use of OpenCV's `fastNlMeansDenoisingColored`:

In [ ]:
def denoise_image(image, dst=None, h=10, hColor=10, templateWindowSize=9, searchWindowSize=21):
    image = (sharpen_image(image, 8) * 255).astype(np.uint8) # cv2 needs image to be of type CV_8UC3 or CV_8UC4 for denoising, so we need to convert it
    
    dst = cv2.fastNlMeansDenoisingColored(image, dst, h, hColor, templateWindowSize, searchWindowSize)

    return dst

#### Measuring noise with PSNR
A high $PSNR$ indicates low noise and high quality, whereas a low $PSNR$ indicates high noise and low quality. However, high $PSNR$ values also indicate a high similarity between two images, because the PSNR value is infinite for identical images. This may lead to a problem because high $PSNR$ values also indicate a low denoising performance if the output image is very similar compared to the noisy input image. Therefore, visual inspection of the denoising performance is essential. To get the $PSNR$ the following definition can be used: $$PSNR=20\cdot\log_{10}\left(\frac{\text{MAX\_VALUE}_I}{\sqrt{\text{MSE}}}\right)$$ 
Where 
- $\text{MAX\_VALUE}_I$ is the maximum possible pixel-value (in this case `255` since we don't normalize or standardize)
- $MSE$ is the **Mean Squared Error** which is defined in the next block(s)

In [ ]:
def get_psnr(original_image, denoised_image, R=255):
    assert original_image.dtype == denoised_image.dtype

    mse = np.mean((original_image.astype(np.float32) - denoised_image.astype(np.float32)) ** 2)
    if mse == 0:
        return np.inf
    
    psnr = 20 * np.log10(R / np.sqrt(mse))
    return psnr

#### Measuring noise with just the MSE
The **Mean Squared Error** (MSE) is a metric that quantifies the difference between two images, in our case the original and a noisy version. When measuring noise, the $MSE$ calculates the average squared differences between corresponding pixels in the two images. A higher $MSE$ indicates a greater deviation from the original, while a lower $MSE$ means the images are more similar. The $MSE$ is defined as $$MSE=\frac{1}{N}(I-I')^2$$ when looking at the images as a matrix of pixel values or therefore $$MSE=\frac{1}{N\times M}\sum_{i=1}^N\sum_{j=1}^M(I_{i,j}-I'_{i,j})^2$$ when looking at the operation over all pixels. In this notation $I$ would be the original image and $I'$ the altered one (denoised) and $\{N,M\}$ define the resolution.

In [ ]:
def get_mse(original_image, denoised_image):
    assert original_image.dtype == denoised_image.dtype

    mse = np.mean((original_image.astype(np.float32) - denoised_image.astype(np.float32)) ** 2)
    return mse

#### Experiments of Noise

In [ ]:
def experiment_noise(image, filter_strength=0.5, n_steps=4):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')
    plt.show()

    processed_images = []
    metrics = {
        'psnr': [],
        'mse': []
    }

    titles = ['PSNR', 'MSE']
    ylabels = ['PSNR Value (dB)', 'MSE Value']
    for i in range(n_steps):
        print(f'Applying Denoising at step {i+1}/{n_steps}')
        denoised = cv2.fastNlMeansDenoisingColored(image, None, i*filter_strength, i*filter_strength, 7, 21)
        processed_images.append(denoised)
        psnr = get_psnr(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.cvtColor(denoised, cv2.COLOR_BGR2GRAY))
        mse = get_mse(image, denoised)
        metrics['mse'].append(mse)
        metrics['psnr'].append(psnr)

    images_fig, ax = plt.subplots(1, n_steps, figsize=(20, 4))
    images_fig.suptitle('Images at each Denoising Step', fontsize=16)

    for i in range(n_steps):
        rgb_image = cv2.cvtColor(processed_images[i], cv2.COLOR_BGR2RGB) 
        ax[i].imshow(rgb_image, cmap='gray')
        ax[i].axis('off')
        ax[i].set_title(f'Step {i+1}', fontsize=12)

    plt.show()

    metrics_fig, ax = plt.subplots(1, 2, figsize=(20, 5))
    metrics_fig.suptitle('Denoising Metrics', fontsize=16)

    for i, (key, values) in enumerate(metrics.items()):
        ax[i].plot(range(n_steps), values, label=f'{key.capitalize()}', color='b')
        ax[i].set_xlabel('Step')
        ax[i].set_title(titles[i], fontsize=12)
        ax[i].set_ylabel(ylabels[i])
        ax[i].set_xticks(range(n_steps))
        ax[i].grid('on')

##### Experiment 1
We are now a bit further south, near the town of Umeå, at the [Norrfors Rapids](https://maps.app.goo.gl/HSHeZjyHLZzyVV5C6). Archeology students of the Umeå University have found these old viking carvings in the year 1984 at the Ume River in Norrfors. The recorded image shows a lot of structure so let's see how our metrics perform on such an image.

In [ ]:
norrfors = cv2.imread(IMAGE_FOLDER+'norrfors.jpeg', cv2.IMREAD_COLOR)

experiment_noise(norrfors, filter_strength=10, n_steps=6)

Both the PSNR and MSE seem to calculate a noticable change in the noise level. The $PSNR$ (Peak Signal-to-Noise Ratio) seems to form a negative exponential curve while the $MSE$ seems to form an S-shaped curve.

##### Experiment 2
Not only the vikings were something to be feared of in the Umeå region. The wide range of different wildlife in the northern part of Sweden allow for a very diverse fauna. The [Brown Bear](https://www.wikiwand.com/en/Eurasian_brown_bear) (or Brunbjörn) is actually a very shy animal in Sweden, if you ever encounter one in the wilderness - you are a lucky person!
The image at hand here is really noisy, so we should see a very noticable change in our metrics and denoising steps.

In [ ]:
bear = cv2.imread(IMAGE_FOLDER+'brown_bear.jpg', cv2.IMREAD_COLOR)

experiment_noise(bear, filter_strength=10, n_steps=6)

This time, we can see the same trend in our metrics but both the $PSNR$ and $MSE$ seem to start to converge at around step `2`. When we look at the images at each step, we could explain that the convergence might emerge because the `cv2`-function we make use of (`fastNlMeansDenoisingColored`) starts to take off a lot of detail at around that step.

To get a more critical view, as described, the $PSNR$ should actually increase when denoising a signal. Yet, in both examples portrayed here, the $PSNR$ decreases through the denoising steps. To note, the $PSNR$ doesn't really go hand-in-hand with the human perception. While we might see a decrease in noise, the denoising algoirthm that I used (`fastNlMeansDenoisingColored`) smoothes the images to a certain extent and that means we lose details but also noise. If $PSNR$ is an objectively good measure of image quality therefore remains an open question or at least a very subjective one.

## Different color spaces (correction or transformations)

#### Measuring color transformations with $\Delta E$
$\Delta E$ is a single number that represents the 'distance' between two colors. The simplest version is the **CIE76** formula which is just the **Euclidean distance** in **Lab space**:
There are more advanced formulas for $\Delta E$ like **CIE94** and **CIEDE2000** which give better results in terms of human color perception, we won't take a closer look at those in this section as they require different packages to convert into different color spaces. To keep it simple, we just look at the lab space that `cv2` allows us to convert to.

Measuring the Euclidean distance in the Lab space roughly measures the same difference that we as humans perceive.

##### The Lab Color Space
![Lab Color Space](./images/notebook_images/lab_color_space.webp)

The Lab Color Space is spanned up in a 3-dimensional sphere. Calculating the Euclidean distance ($\Delta E$) is therefore defined as: $$ \Delta E = \sqrt{(L_1-L_2)^2+(a_1-a_2)^2+(b_1-b_2)^2} $$ Where
- $L_1$ and $L_2$ are the Brightness Value of the input image ($1$) and the transformed image ($2$)
- $a_1$ and $a_2$ are the **Red** and **Green** values for the input image ($1$) and the transformed image ($2$)
- $b_1$ and $b_2$ are the **Blue** and **Yellow** values for the input image ($1$) and the transformed image ($2$)

In [ ]:
def compute_average_deltaE(img1, img2):
    # Convert BGR images to Lab color space
    lab1 = cv2.cvtColor(img1, cv2.COLOR_BGR2Lab)
    lab2 = cv2.cvtColor(img2, cv2.COLOR_BGR2Lab)
    
    # Compute Delta E for each pixel
    deltaE = np.sqrt(np.sum((lab1 - lab2) ** 2, axis=2))
    
    # Return average Delta E
    return np.mean(deltaE)

#### Looking at Color Spaces
To know what kind of transformation is needed for a color or channel, this helper function visualizes the histograms of each color.

In [ ]:
def show_color_space(image, color_space='RGB'):
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[0].axis('off')
    axes[0].set_title('Original Image')

    match color_space:
        case 'BGR':
            channels = ['Blue', 'Green', 'Red']
            for i, ax in enumerate(axes[1:4].flat):
                ax.hist(image[:, :, i].ravel(), bins=255, color=channels[i].lower())
                ax.set_title(f'{channels[i]} Channel')
                ax.set_xlim([0, 255])
                ax.set_ylim([0, 10**4.4])
            plt.show()
            
        case 'HSV':
            image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            channels = ['Hue', 'Saturation', 'Value']
            for i, ax in enumerate(axes[1:4].flat):
                ax.hist(image_hsv[:, :, i].ravel(), bins=255)
                ax.set_title(f'{channels[i]} Channel')
                ax.set_xlim([0, 255])
                ax.set_ylim([0, 10**4.4])
            plt.show()
        case _:
            raise ValueError(f'Color space {color_space} not supported, use either BGR or HSV')

Converting the image to the HSV (Hue, Saturation, Value) color space as it will allow us to manipulate certain colors in an easier way.

The histogram for the hue value shows a great dominance in the spectrum of 25 up to 50 - This corresponds to the hue for the color green, which is very dominant in the northern lights swirl which we are editing.

#### Change color in masked area

**Note**: The `opencv` package uses the Hue Value Range of `0` to `180`, so the function below takes care of the `src_hue` and `target_hue` by dividing it by `2` so we can intuitively use the circular interpretation of the more geometrically correct $2\pi$-Range (`0` to `360` degrees).

In [ ]:
def shift_hue(img, src_hue, target_hue):
    src_hue, target_hue = src_hue/2, target_hue/2

    if img.shape[2] == 4:
        alpha = img[:, :, 3]
    else:
        alpha = None
    
    bgr = img[:, :, 0:3]
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    
    diff_color = target_hue - src_hue
    
    hnew = np.mod(h + diff_color, 360).astype(np.uint8)
    hsv_new = cv2.merge([hnew, s, v])
    bgr_new = cv2.cvtColor(hsv_new, cv2.COLOR_HSV2BGR)
    
    if alpha is not None:
        bgra = cv2.cvtColor(bgr_new, cv2.COLOR_BGR2BGRA)
        bgra[:, :, 3] = alpha
    
    _, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax[0].set_title('Original Image')
    ax[0].axis('off')
    ax[1].imshow(cv2.cvtColor(bgr_new, cv2.COLOR_BGR2RGB))
    ax[1].set_title('Modified Image')
    ax[1].axis('off')
    plt.show()

    print('--- Color Space Statistics ---')
    print(f'Delta E: {compute_average_deltaE(img, bgr_new):.2f}')
    
    return bgr_new

##### Experiment 1
Lets revisit our photo that was taken at the first stop in Kiruna - The northern lights. How amazing would it be if we could just decide what color our northern light had? Well, it is possible through manipulating a color space!

First we have to look at how our colors are distributed through looking at our color space's channel histograms.

In [ ]:
show_color_space(northern_lights, color_space='BGR')

The histograms show that most pixel values are concentrated towards the darker end, indicating that the red channel is less intense for a majority of the image. There's a prominent peak towards the mid-intensity values, indicating a significant presence of green in the image. The histogram for the blue channel also has most of its pixel values towards the darker end but not as much as the red channel. This indicates a moderate presence of blue in the image, possibly accounting for the nighttime sky and the cooler hues in the aurora.

In order to change the color of the aurora, we best switch to the **HSV** Space as it separates the color (Hue) and the brightness (Value) aspects of the image in separate channels - Which is essentially what we want since we would only like to change the color but not the brightness of a color. The image below shows the **HSV** Space. It allows for an intuitive interpretation of what's to change:
![HSV Space](./images/notebook_images/hsv_space.png)

The **HSV** Space gives us a separated "Color Wheel" which we can sort of *spin* into our desired color's place.

In [ ]:
show_color_space(northern_lights, color_space='HSV')

The Hue histogram is multi-modal, with several peaks. The most prominent peak is around the 50-70 range, which typically corresponds to the green hues. This confirms the dominant green color of the aurora. The other two channels aren't as important for what we would like to do but we can still interpret a couple of things while we're at it: The saturation channel shows that the majority of pixels is rather unsaturated or *muted* while there still is a peak at around the Saturation Value of 130-140, this could  be the region where the aurora shines in its saturated green. The Value channel also shows that the image is mostly dark, sitting mostly at a value of around 25-70.

Whats important here is to take note again of what was already mentioned when defining the `shift_hue`-Function: The `opencv` package uses a less intuitive range for its Hue Wheel (`0` to `179`). So the green hue wouldn't actually be in the `50`-`70` range but rather in the `100`-`140` range, which is double the one we can interpret above. For the other two channels `opencv` uses the range from `0` to `255`, which is also less intuitive, a more common approach would be to use the decimal range from `0` to `1`.

To now change the hue (color) of the Image to, let's say pink, we can specify the source hue of about `60` (so `120` in the `360` deg range) (as we noted in the HSV histogram interpretation) and the target hue of `160` (so `320` in the `360` deg range), which corresponds to the color pink according to the hue-wheel below:

![HSV Hue Wheel](./images/notebook_images/hsv_wheel.jpeg)

In [ ]:
northern_lights = cv2.imread(IMAGE_FOLDER + 'northern_lights.png', cv2.IMREAD_UNCHANGED)

shifted = shift_hue(northern_lights, 120, 320)

##### Experiment 2

For our second experiment, lets just validate what we did in the first one. In this example we are now visitng a little Stuga (Cottage) in the snow in the middle of Sweden's little town "[Mora](https://maps.app.goo.gl/1kbHgX55zWh5uk5g8)". Sweden is famous for its **red cottages** and I think it would be fun to maybe get in a bit of change, so why not paint it **blue**? Maybe blue doesn't look too good but we can test our idea using the HSV Shifting technique before taking out the paint brush and ruining the neighbourhood's image.

In [ ]:
stuga = cv2.imread(IMAGE_FOLDER + 'stuga_1.jpeg', cv2.IMREAD_UNCHANGED)

shifted = shift_hue(stuga, 0, 200)

To also interpret our $\Delta E$-Statistic we can now see that this change of color has had less of a numerical difference; When we take a peek back onto the **Lab Color Space** we can see that **Blue and Red** are more or less neighbouring colors while **Green and Pink** are almost on the opposite of each other.

# Task 1.2 Signal Properties
Demonstrate following properties:
- Noise
- Phase
- Bandwidth

In the heart of Sweden, Stockholm stands as a beacon of culture and innovation. Just as the city's rich tapestry is woven from diverse threads, signals too possess distinct characteristics that define their essence. Delving into this section, we'll explore the intricate properties of signals—Noise, Bandwidth, and Phase—drawing parallels to the vibrant and multifaceted spirit of Stockholm's culture.

## Noise

### Measuring Noise in Sound with SNR
The **SNR** (Signal-to-Noise Ratio) is a metric used to measure the quality of a reconstructed or compressed signal compared to the original signal. A higher SNR indicates better quality of the reconstructed or compressed signal. The SNR is measured in decibels (dB) - Usually, a SNR of around `40dB` would be a "well reconstructed" signal.

Mathematically the SNR is computed as $$\text{SNR}=10\times \log_{10}\left(\frac{\text{Signal Power}}{\sqrt{\text{Noise Power}}}\right)$$

In essence, the function provides a decibel value indicating the ratio of the signal power to the power of the noise. Higher SNR values typically signify less distortion and better quality, although it's worth noting that human perception of quality doesn't always linearly align with SNR values.

In [ ]:
def calculate_snr(original_signal, noisy_signal):
    noise = noisy_signal - original_signal

    signal_power = np.mean(np.square(original_signal))
    noise_power = np.mean(np.square(noise))

    if noise_power == 0:
        return np.inf

    return 10 * np.log10(signal_power / noise_power)

### First Hypothesis to Denoising: Band Pass Filtering
A first thought that came to mind was that noise hissing can be attributed to low or high frequencies, or even both - So "cutting" just the part with relevant information could modify the signal enough to make it at least *less* noisy.

In this function I implemented a Band Pass Filter by first masking the frequencies between the given `lowcut` and `highcut` range. The mask is then applied to the Fourier Transform of that signal by setting all frequencies outside of that mask to `0` and then taking the real part of the inverse Fourier Transform to switch back to the time domain.

In [ ]:
def bandpass_filter(signal, lowcut, highcut, sr):
    fft_signal = np.fft.fft(signal)
    fft_freqs = np.fft.fftfreq(len(signal), 1/sr)

    mask = ((fft_freqs >= lowcut) & (fft_freqs <= highcut)) | ((fft_freqs <= -lowcut) & (fft_freqs >= -highcut))
    fft_signal[~mask] = 0
    
    filtered_signal = np.real(np.fft.ifft(fft_signal))
    
    print('--- SNR Statistics ---')
    print(f'SNR Value: {calculate_snr(filtered_signal, signal):.2f}dB')

    _, ax = plt.subplots(1, 2, figsize=(20, 5))
    plt.suptitle('Low Pass Denoising', fontsize=15)
    ax[0].plot(np.arange(len(signal)) / sr, signal)
    ax[0].set_title('Original Signal')
    ax[0].set_xlabel('Time')
    ax[0].set_ylabel('Amplitude')
    ax[1].plot(np.arange(len(filtered_signal)) / sr, filtered_signal)
    ax[1].set_title('Filtered Signal')
    ax[1].set_xlabel('Time')

    plt.figure(figsize=(20, 5))
    plt.plot(np.abs(fft_freqs), np.abs(fft_signal))
    plt.title('Fourier Transform of Filtered Signal')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.show()

    print('--- Original Sound ---')
    display(Audio(signal, rate=sr))
    print('--- Low Pass Filtered Sound ---')
    display(Audio(filtered_signal, rate=sr))

### Second Hypothesis to Denoising: Noise Gating using Artificial Noise and its statistics
When playing around with the first hypothesis' Band Pass Filter I was able to already notice a significant change in how the noise became less but the results were not satisfying enough yet - So I did some more research and found out about **Noise Gating**. This method follows these 8 steps:
1. Switch to the spectral frequency domain of the artificial noise clip (using STFT)
2. Compute noise statistics for every frequency component (mean and standard deviation)
3. Calculate threshold as $mean + sensitivity$, anything below this threshold will be considered as noise
4. Just as with the noise, the function performs an STFT on the actual audio signal we want to clean
5. Create a mask for every frequency and time point: if the magnitude of the signal exceeds the previously determined threshold, the mask will have a value of `1` (meaning keep this component); otherwise, it will have a value of `0` (meaning suppress or remove this component).
6. Smoothen mask to avoid abrupt changes in the mask (which can introduce new artifacts and therefore noise)
7. Multiply the smooth mask with the signals spectrum - This process will keep the components of the signal where the mask is `1` and suppress where it's `0`
8. Reconstruct the cleaned spectrum by applying the inverse Short Time Fourier Transform (iSTFT)



In [ ]:
def spectral_gate(noise_audio, signal_audio, n_fft=2048, hop_size=512, sensitivity=1.5, sr=44100, visualize=True):
    window = 'hann'

    _, _, noise_spec = stft(noise_audio, fs=sr, nperseg=n_fft, noverlap=hop_size, window=window)
    magnitude_noise = np.abs(noise_spec)

    mean_noise = np.mean(magnitude_noise, axis=-1)
    std_noise = np.std(magnitude_noise, axis=-1)

    threshold = mean_noise[:, np.newaxis] + sensitivity * std_noise[:, np.newaxis]

    f, t, signal_spec = stft(signal_audio, fs=sr, nperseg=n_fft, noverlap=hop_size, window=window)
    magnitude_signal = np.abs(signal_spec)

    mask = magnitude_signal > threshold

    freq_smooth = 3
    mask = np.apply_along_axis(lambda m: np.convolve(m, np.ones(freq_smooth)/freq_smooth, mode='same'), axis=0, arr=mask)
    time_smooth = 3
    mask = np.apply_along_axis(lambda m: np.convolve(m, np.ones(time_smooth)/time_smooth, mode='same'), axis=-1, arr=mask)

    cleaned_spec = signal_spec * mask
    _, cleaned_audio = istft(cleaned_spec, fs=sr, nperseg=n_fft, noverlap=hop_size, window=window)

    if len(cleaned_audio) < len(signal_audio):
        cleaned_audio = np.pad(cleaned_audio, (0, len(signal_audio) - len(cleaned_audio)))
    elif len(cleaned_audio) > len(signal_audio):
        cleaned_audio = cleaned_audio[:len(signal_audio)]
        
    print('--- SNR Statistics ---')
    print(f'SNR Value: {calculate_snr(cleaned_audio, signal_audio):.2f}dB')
    
    if visualize:
        difference = np.abs(signal_spec) - np.abs(cleaned_spec)
        plt.figure(figsize=(20, 6))

        plt.subplot(1, 3, 1)
        plt.pcolormesh(t, f, np.abs(signal_spec), shading='gouraud')
        plt.title('Original STFT Magnitude')
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')

        plt.subplot(1, 3, 2)
        plt.pcolormesh(t, f, np.abs(cleaned_spec), shading='gouraud')
        plt.title('Filtered STFT Magnitude')
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')

        plt.subplot(1, 3, 3)
        plt.pcolormesh(t, f, difference, shading='gouraud')
        plt.title('Magnitude Difference')
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')

        plt.tight_layout()
        plt.show()

    return cleaned_audio

##### Experiment 1
As we are now in Stockholm, we cannot forget about *ABBA*. The Swedish music group was formed in 1972 in Stockholm and has shaped music and culture all around the globe. Back in the day, radios used to only pick up very noisy signals so the goal of this experiment is to denoise an old recording of *ABBA's Dancing Queen* which was recorded from an old radio station.

After that we will also try to denoise a recording from Stockholm's Tunnelbana Tillkännagivande (Stockholm's Metro Announcement). Here, our goal will be to suppress the metro train's background noises so that we can get a clearer sound of the announcement.

So lets first look at how the proposed first hypothesis with the Band Pass Filter holds up.

In [ ]:
dancing_queen, dancing_queen_sr = librosa.load(AUDIO_FOLDER + 'noisy_dancing_queen.wav')

Audio(dancing_queen, rate=dancing_queen_sr)

In [ ]:
bandpass_filter(dancing_queen, 50, 2000, dancing_queen_sr)

As we can hear, the "filtered" signal is too harsh on cutting off frequencies. There definitely is less noise, as we can see especially in the beginning and end of the "Filtered Signal Plot", but we also get rid of a lot of detail in the clarity of Agnetha's and Anne-Frid's voices.
The chosen parameters of the frequency range $[50Hz, 2000Hz]$ seems to be a fair tradeoff of detail and noise. 

In [ ]:
tunnelbana, tunnelbana_sr = librosa.load(AUDIO_FOLDER+'tunnelbana.mp3')

Audio(tunnelbana, rate=tunnelbana_sr)

The announcing voice says:
- Nästa: T-Centralen
- Byte till tunnelbana mot: Kungsträdgården, Akalla och Hjulsta samt till pendeltåg
- Tänk på avståndet mellan vagn och plattform när du stiger av

Translated this means:
- Next Stop: T-Centralen
- Connecting Trains to: Kungsträdgården, Akalla and Hjulsta and to Main Train Station
- Consider the gap between the wagon and the platform when you disembark

So let's try if we can hear what the voice says in a clearer way now.

In [ ]:
bandpass_filter(tunnelbana, 800, np.inf, tunnelbana_sr)

The audio sample of the metro announcement already works better for our defined experiment when applying a Band Pass Filter. The metro train's noise has a lot of lower frequencies which have to be removed so we almost simulate the feedback of a small speaker by cutting off the *bass frequencies*. Here, I modified the Band Pass Filter's input to imitate a High Pass Filter by setting the frequency range to $[800Hz, \infty]$, this causes the Band Pass Filter to only cut off all frequencies below `800Hz` which is what we want to get rid off the train's sound.

##### Experiment 2

To use the noise gating method we need to artificially imitate the noise on the noisy track - with that, the function tries to remove parts that are in that same artificial noise signal. This function does exactly that by creating a random array of normally distributed numbers between `0` and `1` which is the same length as the sample that we want to denoise and then it removes frequencies outside of the provided `freq_low` and `freq_high` range while inside the frequency domain.

In [ ]:
def create_noise(signal, sr, freq_low, freq_high, softing_factor=0.2):
    noise = np.random.randn(len(signal))
    duration = len(signal)/sr 
    f_low, f_high = freq_low, freq_high  
    t = np.arange(0, duration, 1/sr)
    
    noise = np.random.normal(0, 1, len(t))

    fft_noise = np.fft.fft(noise)
    fft_noise[int(f_high * duration): -int(f_high * duration)] = 0

    noise = np.fft.ifft(fft_noise).real
    return noise * softing_factor / np.max(noise)

In [ ]:
noise = create_noise(dancing_queen, dancing_queen_sr, 8000, 9000, softing_factor=0.2)

cleaned_audio = spectral_gate(noise, dancing_queen, sr=dancing_queen_sr)

print()
print("--- Artificial Noise [WARNING: LOUD] ---")
display(Audio(noise, rate=dancing_queen_sr))

print("--- Cleaned Audio ---")
display(Audio(cleaned_audio, rate=dancing_queen_sr))

When looking at the spectrogram plots we have a hard time seeing any difference as the `spectral_gate` function only did very subtle changes. If we listen to the audio though, we can definitely hear quite a decrease in noise. Also the $SNR$ shows an increase compared to the rather *rudimentary* method of just applying a Band Pass Filter.

In [ ]:
noise = create_noise(tunnelbana, tunnelbana_sr, 20, 200)

cleaned_audio = spectral_gate(noise, tunnelbana, sr=tunnelbana_sr)

print()
print("--- Artificial Noise [WARNING: LOUD] ---")
display(Audio(noise, rate=dancing_queen_sr))

print("--- Cleaned Audio ---")
display(Audio(cleaned_audio, rate=dancing_queen_sr))

And also here, we can see quite a big increase on the metric-side of the measured $SNR$. 

When looking at the spectrograms we can also see that there is a very noticable difference in the low frequency region, indicating the suppression of the metro train's noise.

## Phase
The phase of a signal describes the shift or offset of a waveform relative to a reference point or another waveform. In the context of sinusoidal signals, phase indicates how much a wave is shifted horizontally along the time axis.

Mathematically, one can imagine a simple sinusoidal signal being defined as $$A(t)=\sin(2\pi ft+\phi)$$ 
Where
- $A(t)$ is the Amplitude of the signal at a given time $t$
- $f$ is the frequency of the sinusoid
- and $\phi$ is the angle of the phase in radians

Note that this is a oversimplification of the actual signals at hand here in our journey through Sweden. The signals here are way more complex and contain thousands of different sinusoids at different angles and therefore phases.

### Measuring Phase Degrees
To quantify a phase shift in signals, especially in the context of sinusoidal or periodic signals, the measure is typically expressed in angles, either in degrees or radians. 

To label the phase with a change that has taken place, we can use the Phase Difference (i.e. the RMS of the Phase Difference).

Mathematically that means we calculate: $$\text{RMS of Phase Difference}=\sqrt{\frac{1}{N}\sum^N_{i=1}\left(\angle Y(f_i)-\angle X(f_i)\right)^2}$$
Where $N$ is the number of frequency bins in the signal.

Here, a larger RMS would mean a larger Phase Difference between two signals.

In [ ]:
def compute_rms_degrees(signal1, signal2):
    spectrum1 = np.fft.fft(signal1)
    spectrum2 = np.fft.fft(signal2)
    
    phase1 = np.angle(spectrum1)
    phase2 = np.angle(spectrum2)
    
    phase_diff_radians = phase2 - phase1
    phase_diff_degrees = np.degrees(phase_diff_radians)
    phase_diff_degrees = (phase_diff_degrees + 180) % 360 - 180

    rms = np.sqrt(np.mean(phase_diff_degrees**2))

    return rms

### Measuring Phase Coherence
Phase coherence (or phase synchronization) is a measure that quantifies how consistently the phase difference is across instances or over time. If two signals have a constant phase difference over time or instances, they are said to have a high phase coherence.

The Coherence $C(f)$ at a particular frequency $f$ can be defined as $$C(f)=\frac{|P_{xy}(f)|^2}{P_{xx}(f)\cdot P_{yy}(f)}$$
Where 
- $x(t)$ and $y(t)$ are the two signals to be compared
- $P_{xy}(f)$ is the cross-spectral density of $x$ and $y$
- $P_{xx}(f)$ and $P_{yy}(f)$ are the power spectral densities for both $x$ and $y$ respectively

Since this function calculates the coherence for each frequency bin, we will take the average of all these coherences to get a valid single-valued metric: $$\bar{C(f)}=\frac{1}{N}\sum^N_{i=1}C(f)_i$$

In [ ]:
def compute_coherence(x, y, fs, nperseg=256):
    x_segments = [x[i:i+nperseg] for i in range(0, len(x), nperseg)]
    y_segments = [y[i:i+nperseg] for i in range(0, len(y), nperseg)]
    
    if len(x_segments[-1]) != nperseg or len(y_segments[-1]) != nperseg:
        x_segments = x_segments[:-1]
        y_segments = y_segments[:-1]
    
    X = [np.fft.fft(seg) for seg in x_segments]
    Y = [np.fft.fft(seg) for seg in y_segments]
    
    Pxx = np.mean([np.abs(xi)**2 for xi in X], axis=0)
    Pyy = np.mean([np.abs(yi)**2 for yi in Y], axis=0)
    Pxy = np.mean([xi*np.conj(yi) for xi, yi in zip(X, Y)], axis=0)
    
    Cxy = np.abs(Pxy)**2 / (Pxx * Pyy)
    
    return np.mean(Cxy[:nperseg//2])

#### Phase shift by angle
The `phase_shift` function applies a phase shift to a given signal within a specified frequency range. Usually a phase shift wouldn't result in a different sounding signal, though by only changing the phase of a specific frequency range the sound also changes. 

In essence, this function takes a signal, identifies the portions of its frequency content that lie within a specified range, and shifts only the phase of those portions by a given amount in degrees. It then provides the modified signal as output. This is a practical demonstration of the phase concept we discussed earlier: the function allows for manipulation of the phase information of a signal in a controlled manner.

In [ ]:
def phase_shift(signal, rate, shift_degrees, fmin=None, fmax=None):
    spectrum = np.fft.fft(signal)

    num_samples = len(signal)
    frequencies = np.fft.fftfreq(num_samples, d=1/rate)
    phase_shift_rad = shift_degrees * np.pi / 180
    shift = np.exp(1j * phase_shift_rad)

    if fmin is None and fmax is None:
        mask = np.ones_like(frequencies, dtype=bool)
    else:
        mask = np.logical_and(frequencies >= fmin, frequencies <= fmax)

    shifted_spectrum = spectrum.copy()
    shifted_spectrum[mask] *= shift

    shifted_signal = np.fft.ifft(shifted_spectrum).real

    return shifted_signal

#### Phase Shift by time
This method takes a more classical approach at phase manipulation. It changes the phase of the entire signal at once, so we will only notice a time shift difference instead of a difference in how the signal sounds.

The sinusoids in a Fourier Transform can also be defined as $e^{\frac{2\pi i n x}{P}}$, where $P$ is the period length. This is possible since $\sin(\phi)=\frac{e^{i\phi}-e^{-i\phi}}{2i}$. As we can see, this introduces the complex identity $i$ to the equation which just simplifies the Fourier Series and reduces the number of terms. Through this we also add a new shifting term to our spectrum by multiplying the new complex term with our spectrum (Fourier Transform of the signal).

In [ ]:
def time_shift(signal, rate, shift_seconds):
    spectrum = np.fft.fft(signal)
    
    num_samples = len(signal)
    frequencies = np.fft.fftfreq(num_samples, d=1/rate)
    phase_shift_rad = 2 * np.pi * frequencies * shift_seconds

    # Apply the phase shift
    shift = np.exp(1j * phase_shift_rad)
    shifted_spectrum = spectrum * shift

    shifted_signal = np.fft.ifft(shifted_spectrum).real

    if shift_seconds > 0:
        shifted_signal_truncated = shifted_signal[:-int(rate * shift_seconds)]
    else:
        shifted_signal_truncated = shifted_signal[-int(rate * shift_seconds):]

    return shifted_signal, shifted_signal_truncated

#### Whats the difference?
To clarify with a simple analogy:

The uniform phase shift is like changing the tuning of a musical instrument. All notes from that instrument are altered, but the relative timing between when you play those notes remains the same.

The time shift is like playing a musical sequence a few seconds later or earlier. The notes remain the same, but when they're played changes.

##### Experiment 1 - Små Grodorna
In Sweden, the most looked forward to holiday is "Midsommar". On this holiday the Swedes celebrate the longest day of the year while drinking "Snaps" and eating lots of different types of fish. The most iconic part of this holiday is the "Maypole". Its a wooden pole that people place into their garden. Families gather around the Maypoles and sing songs while running around it - The most common song is "Små Grodorna", a kids song about "small frogs".

In this experiment we try to mimic the sound of old speakers. By shifting specific low frequencies of the song by an angle of $180 deg$, we can achieve and mimic the sound of "bad" or old speakers as they usually have a hard time with lower frequencies.

In [ ]:
grodorna, grodorna_sr = librosa.load(AUDIO_FOLDER+'grodorna.mp3')

print('Små Grodorna without phase shift')
display(Audio(grodorna, rate=grodorna_sr))

shifted_grodorna = phase_shift(grodorna, grodorna_sr, 180, fmin=0, fmax=1000)

print('Små Grodorna with phase shift')
display(Audio(shifted_grodorna, rate=grodorna_sr))

print('--- Phase Metrics & Statistics ---')
print(f'RMS: {compute_rms_degrees(grodorna, shifted_grodorna):.2f}°')
print(f'Average Coherence: {compute_coherence(grodorna, shifted_grodorna, grodorna_sr)*100:.2f}%')

##### Experiment 2 - Fixing Dancing Queen
Our sample of ABBA's Dancing Queen is not only noisy but it also has this annoying empty part at the beginning in which you only can hear noise. We can fix this by time shifting the signal!

In this experiment we try to uniformly shift the phase of the song by a couple of seconds so we can eliminate that empty part of the song.

In [ ]:
dancing_queen, dancing_queen_sr = librosa.load(AUDIO_FOLDER+'dancing_queen.mp3')

dancing_queen_shifted, dancing_queen_shifted_truncated = time_shift(dancing_queen, dancing_queen_sr, 2.2)

plt.figure(figsize=(20, 5))
plt.plot(np.arange(len(dancing_queen_shifted_truncated)) / dancing_queen_sr, dancing_queen_shifted_truncated)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Dancing Queen Time Domain')
plt.show()

print('--- Phase Metrics & Statistics ---')
print(f'RMS: {compute_rms_degrees(dancing_queen, dancing_queen_shifted):.2f}°')
print(f'Average Coherence: {compute_coherence(dancing_queen, dancing_queen_shifted, dancing_queen_sr)*100:.2f}%')

In [ ]:
Audio(dancing_queen_shifted_truncated, rate=dancing_queen_sr)

Through time shifting the phase of all frequencies by a predifined number (here, `2.2 seconds`) we just move the entire signal on the time axis. In this example we did this to get rid of the signals noise parts, where no music was playing. The plot now shows the signal from the actual beginning of the song without the noise-part.

The metrics are obviously not very applicable in this scenario as we are trying to compare two signals that have the same frequencies but at a different time, so naturally we get a very small coherence.

## Bandwidth
The **bandwidth** is the difference between the *highest* and *lowest* frequencies containing a significant portion (often defined somewhat arbitrarily, e.g., frequencies that contain $50\%$ or $90\%$ of the signal's power) of the signal energy. More on that percentage is described in a block after the function of the bandwidth measure.

### Measuring Bandwidth using Power Spectrum
The Power Spectrum shows how the energy of a signal is distributed across different frequencies. It's essentially a plot where the x-axis represents frequencies and the y-axis shows the energy or "power" at each frequency. By examining the Power Spectrum, you can identify which frequencies contain significant energy. The bandwidth is then measured as the range of frequencies where most of this energy is concentrated.

In [ ]:
def measure_bandwidth(signal, sampling_rate, power_percentage=50):
    fft_signal = np.fft.fft(signal)
    fft_freqs = np.fft.fftfreq(len(signal), 1/sampling_rate)
    
    power_spectrum = np.abs(fft_signal)**2
    max_power = np.max(power_spectrum)
    
    mask = power_spectrum >= (max_power * power_percentage / 100)
    bandwidth_freqs = fft_freqs[mask]
    
    lower_cutoff_freq = np.min(np.abs(bandwidth_freqs))
    upper_cutoff_freq = np.max(np.abs(bandwidth_freqs))
    
    bandwidth = upper_cutoff_freq - lower_cutoff_freq

    return bandwidth


In the function provided, the `power_percentage` parameter, set by default to $50\%$, is used as a threshold to identify the significant components of the signal in the frequency domain for the purpose of estimating bandwidth. It defines a level relative to the peak power in the power spectrum, above which the frequency components are considered relevant to the signal’s bandwidth.

#### Why 50%?
The $50\%$ level is used as a common heuristic and is known as the half-power point or $-3 dB$ point. This convention is widely used in signal processing and electrical engineering, especially in filter design and bandwidth measurement, because it corresponds to the frequencies at which the power of the signal falls to half its peak value. It also provides a balanced approach in many cases, avoiding too wide a bandwidth that includes numerous insignificant frequencies while also not being too narrow to miss important frequency components.

### The Band-pass Filter
A bandpass filter allows signals within a specific frequency range (band) to pass through while attenuating signals outside this range. Essentially, it isolates a particular band of frequencies from a wider spectrum. The band of frequencies is defined by a lower and upper cutoff frequency: signals with frequencies between these two values are allowed to pass through, while signals with frequencies outside this range are significantly reduced. In the given task we want to manipulate the bandwidth — By "cutting off" frequencies below and above a certain threshold, we can achieve such a manipulation.

In [ ]:
def bandpass_filter(signal, lowcut, highcut, sampling_rate):
    fft_signal = np.fft.fft(signal)
    fft_freqs = np.fft.fftfreq(len(signal), 1/sampling_rate)

    mask = ((fft_freqs >= lowcut) & (fft_freqs <= highcut)) | ((fft_freqs <= -lowcut) & (fft_freqs >= -highcut))
    fft_signal[~mask] = 0
    
    filtered_signal = np.fft.ifft(fft_signal)
    
    return np.real(filtered_signal)

##### Bandwidth Experiment

In [ ]:
def experiment_bandwidth(original_signal, sampling_rate, lowcut, highcut):
    # Generate the filtered signal
    filtered_signal = bandpass_filter(original_signal, lowcut, highcut, sampling_rate)
    
    # Measure the bandwidth of the filtered signal
    filtered_bandwidth = measure_bandwidth(filtered_signal, sampling_rate)
    
    # Measure the bandwidth of the original signal
    original_bandwidth = measure_bandwidth(original_signal, sampling_rate)
    
    display(Audio(filtered_signal, rate=sampling_rate))
    
    print('--- Bandwidth Experiment Results ---')
    print(f'Original Bandwidth: {original_bandwidth:.2f} Hz')
    print(f'Bandwidth after Band-pass filter applied: {filtered_bandwidth:.2f} Hz')
    print(f'Rate of change: {100 * (filtered_bandwidth - original_bandwidth) / original_bandwidth:.2f}%')


In this experiment we only filter out the frequencies in the spectrum of the **bass** (40Hz to 180Hz). This experiment would find application in audio editing - Maybe you would like to mix the bassline of *Dancing Queen* together with a sample of a different song.

In [ ]:
experiment_bandwidth(dancing_queen, dancing_queen_sr, 40, 180)

## Nyquist-Shannon Theorem
When we convert a continuous signal (like music) into a digital format (like an MP3), we don't record every single tiny piece of it. Instead, we take periodic "samples" or little snapshots of the signal many times a second. The number of snapshots taken every second is what we call the "sampling rate".

Now, to represent the signal accurately in its digital format, we have a rule: **The sampling rate should be at least twice the highest frequency in the signal**. This rule is basically what we call the Nyquist-Shannon Theorem and it ensures we don't miss out on any important details. If we sample any slower, we might misinterpret the signal, leading to errors and distortions.

Consider when you're streaming music. The music was once a continuous sound wave, but now it's stored as digital data, which means it was sampled. The choice of the sampling rate ensures that what you hear digitally closely represents the original sound. The same principle applies in digital TVs, radios, and even when doctors take digital readings of heartbeats or brainwaves. The right sampling rate ensures the digital representation is as true to the original as possible.

In [ ]:
def demonstrate_sampling_rate(signal, rate, visualizing_window=(100,200), visualize=False):
    if len(signal.shape) > 1:
        signal = signal[:, 0]

    max_frequency_component = 0.1 * rate
    nyquist_rate = 2 * max_frequency_component

    sampling_rates = [nyquist_rate * 0.5, nyquist_rate, nyquist_rate * 2]

    for sr in sampling_rates:
        sr_val = int(sr)
        indices = list(range(0, len(signal), int(rate / sr_val)))
        sampled_data = signal[indices]
        
        mask = (np.array(indices) >= visualizing_window[0]) & (np.array(indices) <= visualizing_window[1])
        
        if visualize:
            plt.figure(figsize=(10, 4))
            plt.scatter(np.array(indices)[mask] - visualizing_window[0], np.array(sampled_data)[mask], color='red', s=5, label=f"Sampled point at {sr_val} Hz")
            plt.plot(range(visualizing_window[1] - visualizing_window[0]), signal[visualizing_window[0]:visualizing_window[1]], label='Original Signal')
            plt.axhline(0, color='gray', lw=1)
            plt.suptitle('Sampling Rate Demonstration', fontsize=15)
            plt.title(f'Sample Rate at {sr_val} Hz ({(sr/nyquist_rate)*100}% of Nyquist rate)', fontsize=10)

            for i in np.array(indices)[mask]:
                plt.plot([i - visualizing_window[0], i - visualizing_window[0]], [0, signal[i]], color='red', lw=1)

            plt.xlim(0, visualizing_window[1] - visualizing_window[0])
            plt.legend()
            plt.show()

        if sr == nyquist_rate:
            print(f'--- Audio Sampled at {sr_val} Hz (at Nyquist rate) ---')
        else:
            print(f'--- Audio Sampled at {sr_val} Hz ({(sr/nyquist_rate)*100}% of Nyquist rate) ---')

        reconstructed_data = np.repeat(sampled_data, int(rate / sr_val))
        display(Audio(reconstructed_data, rate=rate))


This function is used to demonstrate the Theorem by "reconstructing" the signal using a fraction of the original rate by implementing `np.repeat` - In this context, np.repeat is used to simulate a simple and naive method of "reconstructing" the signal after sampling. It's important to note that this is not how real-world reconstruction usually happens, but it provides an illustrative example for demonstration purposes.

##### Experiment 1
In this experiment we will see how different sampling rates affect the sound of our signal. This time we will take a quick detour just outside of Stockholm - Yes, there are mooses around Stockholm. Not in the city centre, although that would be funny.

In [ ]:
moose_sound, moose_sound_sr = librosa.load(AUDIO_FOLDER+'moose.mp3')

demonstrate_sampling_rate(moose_sound, moose_sound_sr, visualizing_window=(100, 200), visualize=True)

As we can see, if we only take a fraction of the actual sampling rate, the reconstruction has a hard time capturing the entire range of the sound's waveform. If we listen through the different sampling sampling rates we can hear that the original sound gets better and better (less robotic) the higher our sampling rate is.

##### Experiment 2
And to come back to the Dancing Queen signal, we can also demonstrate the sampling rate on a signal that is much larger.

In [ ]:
demonstrate_sampling_rate(dancing_queen, dancing_queen_sr)

The same goes for the Dancing Queen song. We can clearly hear a more robotic version on the audio sample with `5512 Hz` and `11025 Hz`.

# Task 1.3 - Normalization and Standardization
In this chapter we will now leave downtown and proceed further into the southern outskirts of Stockholm, where the families live. After we discussed the theory on **normalization** and **standardization** we will go through two experiments, one of which again includes the kid's song "Små Grodorna". In the latter one we will look at a signal from Stockholm's Skeppsholmen island where the government takes record of the yearly average of the sea level.

In signal processing, as well as in many other fields involving data analysis, normalization and standardization are both techniques used to scale and transform data. The concepts are relatively simple, but they serve different purposes. Let's get to know these techniques better.

## Normalization
Normalization typically involves scaling the data so that it fits within a certain range, often between `0` and `1`, or `-1` and `1` or any other set boundaries. The main goal of normalization is to change the values of numeric columns in the data set to a common scale, without distorting differences in the ranges of values. It is particularly useful when features (or signals) have different ranges and you want to ensure they contribute equally to any analysis.

- For a **1-dimensional Signal**, **normalization** is defined as $$s(t)_{\text{normalized}}=\frac{s(t)-min(s(t))}{max(s(t))-min(s(t))}$$ where $s(t)$ is the signal in the time domain.

- For a **2-dimensional Signal**, **normalization** is defined as $$M_{\text{normalized},i,j}=\frac{M_{i,j}-min(M)}{max(M)-min(M)}$$ where $M$ is the 2-dimensional signal (pixel matrix).

If the normalized signal should be bound to a given range, the **normalized signal** has to be further processed as $$s(t)_{\text{scaled}}=s(t)_{\text{normalized}}\cdot (b-a)+a$$ or for 2D Signals as $$M_{\text{scaled}}=M_{\text{normalized}}\times (b-a)+a$$ Where $[a,b]$ defines the range of normalization.

To put this theory into code, this function does exactly what we just defined:

In [ ]:
def normalize_signal(signal, target_range=(0, 1)):
    signal = np.array(signal)
    
    s_min, s_max = np.min(signal), np.max(signal)
    
    normalized = (signal - s_min) / (s_max - s_min)
    
    range_min, range_max = target_range
    normalized = normalized * (range_max - range_min) + range_min
    
    return normalized

## Standardization
Standardization, on the other hand, transforms data to have a mean of 0 and a standard deviation of 1. The process often involves the subtraction of the mean (centering) and then dividing by the standard deviation. Standardization is useful when features (or signals) have different units or different variances, as it makes the data unit-less and scales the variance to 1. This is particularly important for algorithms that are sensitive to the magnitude/variance of features, like SVMs or PCA.

For a **1-dimensional signal**, **standardization** is defined as $$s(t)=\frac{s(t)-\mu}{\sigma}$$ where $\mu$ is the *mean* of the signal and $\sigma$ is the *standard deviation*. 

And similar to the pattern in normalization, the **standardization for a 2D signal** is the same, except $s(t)$ becomes $M$, where each pixel $i,j$ has to be standardized.

And also here, after the mathematical definition, the programmatic definition follows as:

In [ ]:
def standardize_signal(signal, mean=None, std=None):
    signal = np.array(signal)
    
    mean = np.mean(signal) if mean == None else mean
    std = np.std(signal) if std == None else std
    
    standardized = (signal - mean) / std
    
    return standardized

## Testing the defined functions

In [ ]:
def experiment_std_norm(signal, sr=None, standardize=False, normalize=False, normalization_range=(0, 1), axis_labels=['Time', 'Amplitude'], grid=False):
    signal = np.array(signal)
    if standardize:
        signal_standardized = standardize_signal(signal)
    if normalize:
        signal_normalized = normalize_signal(signal, target_range=normalization_range)
    if not standardize and not normalize:
        raise ValueError('You must set either standardize or normalize to `True`')
    

    if len(signal.shape) == 1:
        if normalize:
            _, ax = plt.subplots(1, 2, figsize=(20, 5))
            ax[0].plot(np.arange(len(signal)), signal)
            ax[0].set_title('Original Signal')
            ax[0].set_xlabel(axis_labels[0])
            ax[0].grid(grid)
            ax[0].set_ylabel(axis_labels[1])
            ax[1].plot(np.arange(len(signal_normalized)), signal_normalized)
            ax[1].set_ylabel(f'{axis_labels[1]} normalized')
            ax[1].set_xlabel(axis_labels[0])
            ax[1].set_title('Normalized Signal')
            ax[1].grid(grid)
            plt.show()

            if sr:
                print('--- Original Audio Signal ---')
                display(Audio(signal, rate=sr))
                print('--- Normalized Audio Signal ---')
                display(Audio(signal_normalized, rate=sr))
        if standardize:
            _, ax = plt.subplots(1, 2, figsize=(20, 5))
            ax[0].plot(np.arange(len(signal)), signal)
            ax[0].set_title('Original Signal')
            ax[0].set_xlabel(axis_labels[0])
            ax[0].grid(grid)
            ax[0].set_ylabel(axis_labels[1])
            ax[1].plot(np.arange(len(signal_standardized)), signal_standardized)
            ax[1].set_ylabel(f'{axis_labels[1]} standardized')
            ax[1].set_xlabel(axis_labels[0])
            ax[1].set_title('Standardized Signal')
            ax[1].grid(grid)
            plt.show()

            if sr:
                print('--- Original Audio Signal ---')
                display(Audio(signal, rate=sr))
                print('--- Standardized Audio Signal ---')
                display(Audio(signal_standardized, rate=sr))

    if len(signal.shape) == 2:
        if normalize:
            _, ax = plt.subplots(1, 2, figsize=(20, 5))
            ax[0].imshow(signal, cmap='gray')
            ax[0].axis('off')
            ax[0].set_title('Original Image')
            ax[1].imshow(signal_normalized, cmap='gray')
            ax[1].axis('off')
            ax[1].set_title('Normalized Image')
            plt.show()
        
        if standardize:
            _, ax = plt.subplots(1, 2, figsize=(20, 5))
            ax[0].imshow(signal, cmap='gray')
            ax[0].axis('off')
            ax[0].set_title('Original Image')
            ax[1].imshow(signal_standardized, cmap='gray')
            ax[1].axis('off')
            ax[1].set_title('Standardized Image')
            plt.show()

##### Experiment 1
The goal of this experiment is to show how the signal of "Små Grodorna" changes after normalization.

In [ ]:
experiment_std_norm(shifted_grodorna, sr=grodorna_sr, normalize=True, standardize=True)

As we can see normalization and standardization does not alter the shape of our signal or what sounds our piece plays back but we can see a difference in the amplitude - So depending on our defined normalization range or the signals $\mu$ (mean) or $\sigma$ (standard deviation), it can get louder or quieter. 

##### Experiment 2
To not only look at audio signals, this experiment demonstrates the **standardization** and **normalization** of the the yearly averaged water level from 1889 until 2022 in Stockholm.

In [ ]:
water_levels = read_csv(DATA_FOLDER+'havsniva_ANN_stockholm_skeppsholmen_adjust.csv', delimiter=';')

experiment_std_norm(water_levels['Annual mean value'], standardize=True, normalize=True, axis_labels=['Year', 'Water Level (cm)'], grid=True)

And the same interpretation applies here. In normalization the signal is being clipped into a given range, here $[-1, 1]$ and in standardization the signal is being clipped into a range with standard deviation of $1$ and a mean of $0$.

# Task 2.1 - Filtering in the spatial domain
Create a filter that:
- sharpens
- distorts

### Filtering functions
This first filtering algorithm can apply a kernel to a 1-dimensional signal, such as audio.

- It first pads the signal to handle edge cases (using `np.pad`)
- Then slides the kernel across the signal, multiplying and summing values to produce the output 

The result is a new signal that's been filtered by the kernel.

In [ ]:
def filter_1d(signal, kernel):
    signal_len = len(signal)
    kernel_len = len(kernel)
    
    pad_len = kernel_len // 2
    
    padded_signal = np.pad(signal, (pad_len, pad_len), mode='constant', constant_values=0)
    
    output = np.zeros(signal_len)
    
    for i in range(signal_len):
        output[i] = np.sum(padded_signal[i: i+kernel_len] * kernel[::-1])  # kernel is reversed for convolution
        
    return output

The `filter_2d` function applies a 2D convolution between an input image and a given kernel. It also first pads the image and then produces a filtered output. If `clip_output` is set, the output values are limited to a range between `0` and `255`. The `apply_2d_filter` function uses `filter_2d` to handle both grayscale and color images, applying the convolution separately to each channel in color images - This has to be done since `filter_2d` only handles a single channel.

In [ ]:
def filter_2d(image, kernel, clip_output=True):
    img_height, img_width = image.shape[0], image.shape[1]
    k_height, k_width = kernel.shape[0], kernel.shape[1]
    
    pad_height = k_height // 2
    pad_width = k_width // 2
    
    padded_img = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant', constant_values=0)
    
    output = np.zeros((img_height, img_width))
    
    for i in range(img_height):
        for j in range(img_width):
            output[i, j] = np.sum(padded_img[i: i+k_height, j: j+k_width] * kernel)
    
    if clip_output:
        output = np.clip(output, 0, 255)
    
    return output

def apply_2d_filter(image, kernel, clip_output=True):
    if len(image.shape) == 2:
        return filter_2d(image, kernel)
    elif len(image.shape) == 3:
        output = np.zeros_like(image)
        for channel in range(3):
            output[:, :, channel] = filter_2d(image[:, :, channel], kernel, clip_output=clip_output)
        return output
    else:
        raise ValueError("Invalid image format.")

### Helpers
This subchapter holds all the helper-functions. Mostly those are functions that outsource the creation of a filtering kernel since such kernels can change in their nature very much based on the task at hand.

#### Gaussian Kernel
- The function first creates a linear space `ax` that represents the distance of each pixel in the kernel from the center.
- Then, the Gaussian function is applied to this linear space, resulting in a 1D Gaussian array gauss. The Gaussian function is given by: $$f(x)=e^{-\frac{1}{2}(\frac{x}{\sigma})^2}$$

The outer product of `gauss` with itself (using `np.outer(gauss, gauss)`) creates a 2D Gaussian kernel. This operation effectively applies the Gaussian function in both the `x` and `y` dimensions.

In [ ]:
def create_gaussian_kernel(length=5, sig=None):
    assert length % 2 == 1, 'Kernel length must be odd'

    if not sig: sig = length // 6 # sigma default, best practice

    ax = np.linspace(-(length - 1) / 2., (length - 1) / 2., length)
    gauss = np.exp(-0.5 * np.square(ax) / np.square(sig))
    kernel = np.outer(gauss, gauss)
    return kernel / np.sum(kernel)

The created kernel is a Gaussian kernel. In image processing, Gaussian kernels are used primarily for blurring or smoothing operations. The Gaussian kernel gives more weight to the pixels closer to the center of the window (the pixel being processed), and this weight decreases for pixels further from the center. This results in a weighted average where the central pixels contribute more to the final value than the distant pixels, leading to a blurring effect.

Notes and observations:
- larger kernels add a more noticable effect
- larger kernels use more compute
- $n_x * n_y$ (kernel area) operations are made per pixel

#### Fundamental Frequency
The `compute_fundamental_frequency` function calculates the fundamental frequency of a given time-domain signal by converting it to the frequency domain using FFT and then identifying the frequency with the highest magnitude in the positive frequency domain. This frequency corresponds to the pitch or fundamental frequency of the signal.

In [ ]:
def compute_fundamental_frequency(signal, sample_rate):
    signal_fft = np.fft.fft(signal)
    frequencies = np.fft.fftfreq(len(signal), 1/sample_rate)

    positive_freq_domain = frequencies > 0
    peak_idx = np.argmax(np.abs(signal_fft[positive_freq_domain]))

    fundamental_freq = frequencies[positive_freq_domain][peak_idx]

    return fundamental_freq

#### Laplacian Kernel / Sharpening Kernel
By utilizing this function we can create a sharpening kernel. In image processing, sharpening kernels are used to enhance the edges and fine details in an image.

For a 2D kernel, the negative values surrounding the center emphasize the difference between a pixel and its neighbors, while the positive center value amplifies the pixel itself. This results in a sharpening effect. The 1D kernel works similarly but along one dimension.

A common sharpening Kernel looks like this: $$\begin{bmatrix}0 & -1 & 0 \\ -1 & 5 & -1 \\ 0 & -1 & 0\end{bmatrix}$$

This is also the kernel I use to create a new sharpening kernel, though by changing the following functions parameters, the classic sharpening kernel gets amplified by some degree.

In [ ]:
def create_sharpening_kernel(dimensions=1, length=3, negative_magnitude=1, positive_magnitude=1, normalize_kernel=True):
    assert dimensions > 0 and dimensions < 3, 'Dimensions must be either 1 or 2'

    if dimensions == 1:
        kernel = -1*np.ones(length) * negative_magnitude
        kernel[length // 2] = positive_magnitude
    
    elif dimensions == 2:
        kernel = -1*np.ones((length, length)) * negative_magnitude
        kernel[length // 2, length // 2] = 5*positive_magnitude
        kernel[0, 0] = 0
        kernel[0, -1] = 0
        kernel[-1, 0] = 0
        kernel[-1, -1] = 0

    if normalize_kernel:
        kernel = kernel / np.sum(kernel)
        return kernel
    else:
        return kernel

## Sharpness
Sharpness in audio or any 1D signal pertains to the rapid changes or transitions in the signal's amplitude. A "sharp" audio signal might have quick peaks and troughs, implying rapid transitions between high and low values. Mathematically, this can be gauged by the magnitude of the derivative of the signal. A higher average magnitude indicates more sharpness. Root Mean Squared Slope (RMSS) or other similar metrics can quantify this.

Sharpness in images relates to the clarity of edges and fine details. A sharp image has well-defined boundaries between objects and clear textures. Blurry or out-of-focus images are considered less sharp. Mathematically, this is often captured by the magnitude of the image gradient (how pixel values change from one pixel to its neighbors). A high average gradient magnitude across an image indicates more sharpness.

### Sharpness of 1D-Signal
**What sharpness is in simpler terms for 1D signals**: Imagine a calm Swedish lake (en lugn svensk sjö) with gentle waves as a smooth audio signal. A stormy sea with rapid and high waves is akin to a sharp audio signal.

#### Measuring 1D-Sharpness using Root Mean Square Slope (RMSS)
The function `compute_rmss` calculates the Root Mean Squared Slope (RMSS) of a signal. The RMSS is a metric that quantifies the average rate of change or the slope of a signal. We can expect a signal to be "sharp" if its RMSS is high - That would describe a rather steeply jagged signal pattern.

Mathematically this metric is defined as $$\text{RMSS}=\sqrt{\frac{1}{N}\sum_{i=1}^N (\Delta s_i)^2}$$ Where $\Delta s_i$ is the rise (i.e. rise over run), or mathematcially $\Delta s_i = s_{i+1}-s_i$. This is equivalent to the slope since the "run" (or time difference on the x-axis) is consistently 1 unit in discrete signals, so the rise becomes the slope.

In [ ]:
def compute_rmss(signal):
    derivative = np.diff(signal)
    squared_derivative = np.square(derivative)
    mean_squared_derivative = np.mean(squared_derivative)
    
    rmss = np.sqrt(mean_squared_derivative)
    
    return rmss

#### Adding 1D-Sharpness with a common sharpening kernel
This function just packages the kernel calculation and appliance of that kernel in successive order together.

In [ ]:
def apply_1d_sharpness(signal, sharpness_factor=2):
    sharpening_kernel_1d = create_sharpening_kernel(dimensions=1, 
                                                    length=3, 
                                                    negative_magnitude=sharpness_factor, 
                                                    positive_magnitude=sharpness_factor)

    sharpened_signal = filter_1d(signal, sharpening_kernel_1d)

    return sharpened_signal

### Sharpness of 2D-Signal
**What sharpness is in simpler terms for 2D signals**: Think of looking through a clear window of a cottage in the woods versus a foggy one. Through the clear window, the edges and textures of the forest's trees outside are distinct and well-defined (sharp). Through the foggy one, everything appears smudged and less clear (less sharp).

#### Measuring 2D-Sharpness using Gradient Sharpness
For this step I will reuse the predefined function `get_gradient_sharpness` from Task 1.1 as it seems to capture the change the best.

#### Adding 2D-Sharpness with a 2D-Sharpening-Kernel
This is just as above a packaging function that holds together the process of applying sharpness.

In [ ]:
def apply_2d_sharpness(image, sharpness_factor=1, normalize_kernel=True):
    sharpening_kernel_2d = create_sharpening_kernel(dimensions=2, 
                                                    length=3, 
                                                    negative_magnitude=sharpness_factor, 
                                                    positive_magnitude=sharpness_factor,
                                                    normalize_kernel=normalize_kernel)
    
    sharpened_image = apply_2d_filter(image, sharpening_kernel_2d)

    return sharpened_image

### Sharpness Experiment of both 1D- and 2D-Signals

In [ ]:
def experiment_audio_sharpness(audio, sr, sharpness_factor=1):
    sharpened_audio = apply_1d_sharpness(audio, sharpness_factor)

    _, ax = plt.subplots(1, 2, figsize=(20, 5))
    ax[0].plot(np.arange(len(audio)) / sr, audio)
    ax[0].set_title('Original Audio')
    ax[0].set_xlabel('Time')
    ax[0].set_ylabel('Amplitude')
    ax[1].plot(np.arange(len(sharpened_audio)) / sr, sharpened_audio)
    ax[1].set_title('Sharpened Audio')
    ax[1].set_xlabel('Time')
    ax[1].set_ylabel('Amplitude')
    plt.show()

    original_rmss = compute_rmss(audio)
    sharpened_rmss = compute_rmss(sharpened_audio)

    print(f'--- Sharpness Experiment Results ---')
    print(f'Original Audio RMSS: {original_rmss:.2f}')
    print(f'Sharpened Audio RMSS: {sharpened_rmss:.2f}')

    display(Audio(sharpened_audio, rate=sr))

In [ ]:
def experiment_image_sharpness(image, n_sharpness_operations=3, normalize_kernel=False):
    print(f'Running experiment with image of shape {image.shape[0]} x {image.shape[1]} and {image.shape[2]} channels')

    sharpened_images = []
    gradient_sharpness = []
    measured_times = []
    for i in range(n_sharpness_operations):
        print(f'Applying sharpness to image {i+1}/{n_sharpness_operations} ...')
        if len(sharpened_images) == 0:
            start_time = time.time()
            sharpened_images.append(apply_2d_sharpness(image, normalize_kernel=normalize_kernel))
            end_time = time.time()
            measured_times.append(end_time - start_time)
        else:
            start_time = time.time()
            sharpened_images.append(apply_2d_sharpness(sharpened_images[-1], normalize_kernel=normalize_kernel))
            end_time = time.time()
            measured_times.append(end_time - start_time)
        gradient_sharpness.append(get_gradient_sharpness(sharpened_images[-1]))

    _, axes = plt.subplots(1, n_sharpness_operations+1, figsize=(15, 5))
    axes[0].imshow(image)
    axes[0].set_title('Original Image')
    axes[0].set_xlabel(f'Gradient Sharpness: {get_gradient_sharpness(image):.2f}')
    for i, ax in enumerate(axes[1:]):
        ax.imshow(sharpened_images[i])
        ax.set_title(f'Image after {i+1} operation(s)')
        ax.set_xlabel(f'Gradient Sharpness: {gradient_sharpness[i]:.2f}')
    plt.show()

    print('--- Sharpness Experiment Results ---')
    print(f'Gradient Sharpness for original image: {get_gradient_sharpness(image):.2f}')
    for i in range(len(gradient_sharpness)):
        print(f'Gradient Sharpness for image after {i+1} operation(s): {gradient_sharpness[i]:.2f}')
    print()
    print(f'--- Sharpness Time Performance Results ---')
    for i in range(len(measured_times)):
        print(f'Time to apply sharpness at position {i+1}: {measured_times[i]:.2f} seconds')
    print(f'Average time to apply sharpness: {np.mean(measured_times):.2f} seconds')
    print(f'Total time taken for all {n_sharpness_operations} sharpness operations: {np.sum(measured_times):.2f} seconds')

##### Experiment 1
First, we will look at the sharpness of audio.

In [ ]:
experiment_audio_sharpness(dancing_queen, dancing_queen_sr, 1)

The applied kernel doesn't change the waveform drastically on the first sight, though if we take a closer look we can see that relatively low amplitudes were filtered to be a little lower while relatively high signals in the original audio got amplified by quite a bit.

When listening to the processed audio we can notice that the snare, high-hat and vocals got fairly loud as they got amplified during sharpening. These instruments are already more on the sharp side when looking at their sound since they build up very fast and fade out very fast.

##### Experiment 2
In this experiment we will closer examinate the results of sharpening the **Nobel Prize** coin. Coins usually have a flat surface so the goal here is to make the coin's engravings more clear and apparent.

In [ ]:
nobel_prize = cv2.imread(IMAGE_FOLDER + 'nobel_prize.png', cv2.IMREAD_COLOR)
nobel_prize = cv2.cvtColor(nobel_prize, cv2.COLOR_BGR2RGB)

experiment_image_sharpness(nobel_prize, 3, normalize_kernel=False)

Throughout the sharpening process we can see that [Alfred Nobel](https://www.wikiwand.com/en/Alfred_Nobel)'s side-profile gets more apparent as we apply more sharpness. Though there also is a point where *too much* sharpness is applied, in our case this would be between Step 2 and 3. Between those steps the kernel starts to amplify noise, which is not the desired product.

##### Experiment 3
The last experiment should show if the kernel-filtering function that was defined for 2-dimensional signals shows any difference between filtering relatively small images versus relatively large (high resolution) images. For that we will compare the measured time in Experiment 2 with the measured time of the following sharpening process.

In [ ]:
albino_moose = cv2.imread(IMAGE_FOLDER + 'albino_moose.jpeg', cv2.IMREAD_COLOR)
albino_moose = cv2.cvtColor(albino_moose, cv2.COLOR_BGR2RGB)

experiment_image_sharpness(albino_moose, 3, normalize_kernel=False)

In the last experiment we can see that the kernel takes much longer to be applied. The image at hand is about three times as big:

Number of Pixels for the **first** image: $N_{\text{pixels}}=580^2=336400$

Time to apply kernel for the **second** image: $\approx 7.8 s$

Number of Pixels for the **second** image: $N_{\text{pixels}}=801\cdot 1200=961200$

Time to apply kernel for the **second** image: $\approx 21.0 s$

Note: These recordings were ran on an ARM64 Chip (Apple M1)

These statistics speak for a linear correlation of the **number of pixels** and the **processing time**. When looking at the different functions, here are the time complexities for each function:
- `filter_1d(signal, kernel)`: $O(\text{signal\_len} \times \text{kernel\_len})$
- `filter_2d(image, kernel, clip_output=True)`: $O(\text{image\_height} \times \text{image\_width} \times \text{kernel\_height} \times \text{kernel\_width})$ for a single-channel image (grayscale) and $3 \times  O(\text{image\_height} \times \text{image\_width} \times \text{kernel\_height} \times \text{kernel\_width})$ for a 3-channel image, such as RGB or BGR


## Distortion
Distortion in signals refers to any deviation or alteration from the true or original signal. It means that the output signal differs in some systematic way from the input, even if the integrity of the underlying structure might still be discernible.

### Distortion of 1D-Signal

#### Measuring 1D-Distortion using Total Harmonic Distortion (THD)
Total Harmonic Distortion (THD) is a metric used primarily in audio and power systems to quantify the level of harmonic distortion present in a signal. In simpler terms, it's a measure of how much the output of a system deviates from the desired or input signal due to the presence of harmonics.

Mathematically, if $V_1$ is the $RMS$-Value of the fundamental frequency, and $V_2, V_3, \dots$ are the $RMS$-Values of all the following harmonics then the $THD$ expresses itself mathematically as $$THD=\frac{\sqrt{V_2^2 + V_3^2+\dots}}{V_1}$$

The $THD$ is usually described as a percentage.

In [ ]:
def compute_thd(signal, fs, fundamental_freq, num_harmonics):
    # Compute the FFT of the signal
    signal_fft = np.fft.fft(signal)
    freqs = np.fft.fftfreq(len(signal), 1/fs)

    # Find the closest index to the fundamental frequency
    fundamental_idx = np.argmin(np.abs(freqs - fundamental_freq))

    # Compute power of the fundamental frequency
    p1 = np.abs(signal_fft[fundamental_idx])**2

    # Compute power of the harmonics
    harmonic_powers = [
        np.abs(signal_fft[fundamental_idx * (n+2)])**2 for n in range(num_harmonics)
    ]

    # Compute THD
    thd = np.sqrt(sum(harmonic_powers)) / np.sqrt(p1)
    return thd


#### Adding 1D-Distortion with Impulse Response (IR) acting as a kernel
This function describes a system (or a series of systems) where the input audio is first passed through an amplifier (characterized by its impulse response), then through a distortion unit (the `np.clip` function), and finally being normalized. The impulse response here is straightforward and serves as a mechanism to scale or amplify the original audio signal without altering its shape.

In [ ]:
def create_distorted_audio(original_signal, sample_rate, amplification_factor, clipping_level):
    impulse_response = [amplification_factor]
    amplified_signal = filter_1d(original_signal, impulse_response)

    distorted_signal = np.clip(amplified_signal, -clipping_level, clipping_level)
    distorted_signal /= np.max(np.abs(distorted_signal))

    return distorted_signal

### Distortion of 2D-Signal

#### Measuring 2D-Distortion using Mean Squared Error (MSE)
Mean Squared Error (MSE) is commonly employed to measure the difference between two signals, such as images. When used as a distortion metric for images, $MSE$ quantifies the difference between a reference image and a distorted or processed version of that image, as already pointed out in an earlier chapter where we used to measure the noise of images over time (or steps). 

The $MSE$ is defined as $$MSE(I, I')=\frac{1}{M \times N}\sum_{i=1}^M \sum_{j=1}^N[I(i,j)-I'(i,j)]^2$$ 
Where
- $I$ is the original input image as a matrix of pixel values
- $I'$ is the processed (distorted) image as a matrix of pixel values
- $M$ and $N$ are the width and height of the images
- And therefore $I(i,j)$ and $I'(i,j)$ are the pixel's intensity at position $(i,j)$

In [ ]:
def compute_mse(image1, image2):
    if image1.shape != image2.shape:
        raise ValueError("Input images must have the same shape")
    
    return np.mean((image1 - image2) ** 2)

In Python, by utilizing `numpy` the matrix calculation gets rather simplified.

#### Adding 2D-Distortion by amplifying each pixel

Instead of just applying an amplification factor, like in the 1-dimensional version of the Impulse Response Distortion, here we amplify a pixel by multiplying it with an `amplification_factor` and therefore changing its intensity without taking its neighboring pixels into account.

In [ ]:
def create_distorted_image(image, amplification_factor):
    kernel = np.array([
        [0, 0, 0],
        [0, amplification_factor, 0],
        [0, 0, 0]
    ])

    return apply_2d_filter(image, kernel, clip_output=False)

##### Distortion Experiments of both 1D- and 2D-Signals
**For 1D-Signal (Orchestral Piece)**: The Swedish composer [Ludwig Göransson](https://www.wikiwand.com/en/Ludwig_G%C3%B6ransson) composed this piece for the movie "[Oppenheimer](https://www.wikiwand.com/en/Oppenheimer_(film))". The movie is about the development and research of the atomic bomb during the cold war - Therefore an application of distortion could be for the movie producers to introduce a heavier sound in a scene where such a bomb explodes.  

In [ ]:
def experiment_audio_distortion(original_signal, sampling_rate, distortion_factor=10.0, clipping_level=1.0, num_harmonics=7):
    fundamental_freq = compute_fundamental_frequency(original_signal, sampling_rate)

    distorted_signal = create_distorted_audio(original_signal, sampling_rate, distortion_factor, clipping_level)

    thd_orig = compute_thd(original_signal, sampling_rate, fundamental_freq, num_harmonics)
    thd_dist = compute_thd(distorted_signal, sampling_rate, fundamental_freq, num_harmonics)

    print(f'--- 1D-Signal Distortion Experiment Results ---')
    print(f'Original THD: {thd_orig*100:.2f}%')
    print(f'Distorted THD: {thd_dist*100:.2f}%')
    print('')
    print('--- Distorted 1D-Signal (Audio) [WARNING: LOUD] ---')
    display(Audio(distorted_signal, rate=sampling_rate))

    plt.figure(figsize=(12, 6))
    plt.subplot(2, 1, 1)
    plt.title("Original Audio Signal")
    plt.plot(original_signal)
    plt.subplot(2, 1, 2)
    plt.title("Distorted Audio Signal")
    plt.plot(distorted_signal)
    plt.tight_layout()
    plt.show()

In this experiment I amplify each signal by an amplification factor of 20. The original piece is already on the distorted side, especially at the end of the piece it gets more and more distorted. To minimize clipping I found the clipping level of 4 to be the most effective. The number of harmonics which was used in Ludwig Göranssons Orchestra is derived from a source on the internet.

In [ ]:
oppenheimer, op_sr = librosa.load(AUDIO_FOLDER+'oppenheimer.mp3')

experiment_audio_distortion(oppenheimer, op_sr, 20.0, 4.0, 7)

**For 2D-Signal (Image)**: For this experiment we will look at an image of a typical Swedish Stuga. Usually distortion is a technique used in the context of art. Often times we can also see distorted imagery in movies, when editors want to emphasize a hallucinated or dreamed scene.

In [ ]:
def experiment_image_distortion(image, n_amplifications=6):
    assert n_amplifications > 0, 'There must be more than 0 amplifications made to compare the images with the MSE metric.'

    distorted_images = [image]
    for i in range(n_amplifications-1):
        print(f'Distorting image {i+1}/{n_amplifications-1} ...')
        distorted_images.append(create_distorted_image(distorted_images[-1], 1+i*0.3))

    mse_values = []
    for img in distorted_images:
        mse_values.append(compute_mse(image, img))

    _, axes = plt.subplots(1, n_amplifications, figsize=(20, 5))
    for i, ax in enumerate(axes.flat):
        ax.imshow(distorted_images[i])
        ax.set_title(f'Step {i+1}')
        ax.set_xlabel(f'Amplification Factor: {1+i*0.1:.1f}')
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

    # plot the mse values
    plt.plot(range(n_amplifications), mse_values)
    plt.xlabel('Amplification Step')
    plt.ylabel('MSE')
    plt.title('MSE Values')
    plt.show()


In [ ]:
stuga_bgr = cv2.imread(IMAGE_FOLDER+'stuga_2.jpeg', cv2.IMREAD_COLOR)
stuga = cv2.cvtColor(stuga_bgr, cv2.COLOR_BGR2RGB)

experiment_image_distortion(stuga)

In this experiment, the added distortion makes the image look like the image was taken on a thermal cam (Step 4). Here, the $MSE$ measures a steep change between step `2` and `3` (in the plot, step `1` and `2`). This is a great example to show how the $MSE$ isn't very good when wanting to compare it's values to the human visual perception. We, as humans might say the biggest difference is between step `3` and `4`.

# Task 2.2 - Filtering in the spectral domain
In this task we travel down into the flatlands of Sweden. 

## Implementation of a method that filters signals (1D and 2D) in the spectral domain
The filter_frequencies function filters out specific frequency ranges from a signal. 

Here's how it works:
1. If given a 1D signal (like an audio waveform) and a sampling rate:
- It removes all frequencies within the specified range from the signal.
2. If given a 2D signal (like an image):
- It keeps only the frequencies within a circle defined by the given range in the frequency domain.

In both cases, the function returns the signal with the specified frequencies (`freq_range`) filtered out.

In [ ]:
def filter_frequencies(signal, freq_range, sampling_rate=None):
    signal = signal.astype(float)

    if len(signal.shape) == 1 and sampling_rate != None:
        fft_signal = np.fft.fft(signal)
        frequencies = np.fft.fftfreq(len(signal), 1/sampling_rate)

        mask = (np.abs(frequencies) >= freq_range[0]) & (np.abs(frequencies) <= freq_range[1])
        fft_signal[mask] = 0
        
        filtered_signal = np.fft.ifft(fft_signal).real

    elif len(signal.shape) == 2:
        fft_signal = np.fft.fft2(signal)
        fft_shift = np.fft.fftshift(fft_signal)
        rows, cols = signal.shape
        crow, ccol = rows//2 , cols//2
        
        x = np.arange(0, rows, 1)
        y = np.arange(0, cols, 1)
        x, y = np.meshgrid(y, x)
        distance = np.sqrt((x-ccol)**2 + (y-crow)**2)
        mask = (distance < freq_range[0]) | (distance > freq_range[1])
        
        fft_shift *= mask
        fft_isignal = np.fft.ifftshift(fft_shift)
        filtered_signal = np.fft.ifft2(fft_isignal).real
        
    else:
        raise ValueError("Input signal must be either 1D or 2D.")
    
    return filtered_signal

#### Helper to plot spectral view
In order to make out which frequencies to block out, this function will help us visualize the **Fourier Spectrum** of each signal.

In [ ]:
def plot_fourier_spectrum(original_signal, filtered_signal, sampling_rate=None):
    original_signal, filtered_signal = original_signal.astype(float), filtered_signal.astype(float)
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    if len(original_signal.shape) == 1 and sampling_rate != None:
        fft_orig_signal, orig_frequencies = np.fft.fft(original_signal), np.fft.fftfreq(len(original_signal), 1/sampling_rate)
        fft_filt_signal, filt_frequencies = np.fft.fft(filtered_signal), np.fft.fftfreq(len(filtered_signal), 1/sampling_rate)
        
        axes[0].plot(orig_frequencies, np.abs(fft_orig_signal))
        axes[0].set_title('Original Signal')
        axes[0].set_ylabel('Amplitude')
        axes[0].set_xlabel('Frequency (Hz)')
        axes[1].plot(filt_frequencies, np.abs(fft_filt_signal))
        axes[1].set_title('Filtered Signal')
        axes[1].set_ylabel('Amplitude')
        axes[1].set_xlabel('Frequency (Hz)')
        fig.suptitle('Fourier Spectrum of Original and Filtered Signal')
        
    elif len(original_signal.shape) == 2:
        fft_orig_signal = np.fft.fft2(original_signal)
        fft_orig_shift = np.fft.fftshift(fft_orig_signal)
        fft_filt_signal = np.fft.fft2(filtered_signal)
        fft_filt_shift = np.fft.fftshift(fft_filt_signal)
        
        axes[0].imshow(np.log(np.abs(fft_orig_shift)+1), cmap="inferno")  # Log scaled for better visualization
        axes[0].set_title('Original Image')
        axes[1].imshow(np.log(np.abs(fft_filt_shift)+1), cmap="inferno")  # Log scaled for better visualization
        axes[1].set_title('Filtered Image')
        fig.suptitle('Fourier Spectrum of Original and Filtered Image (Log scaled)')
        
    else:
        raise ValueError("Input signal must be either 1D or 2D.")


## Testing the method for 1D signals
We are going to test the proposed function by analyzing a very common bird in Sweden's wilderness -- The [Great Tit](https://www.wikiwand.com/en/Great%20tit), or in Swedish "den Talgoxe".

In [ ]:
talgoxe, talgoxe_sr = librosa.load(AUDIO_FOLDER+'talgoxe.mp3')
filtered_talgoxe = filter_frequencies(talgoxe, [0,3500], sampling_rate=talgoxe_sr)

plot_fourier_spectrum(talgoxe, filtered_talgoxe, talgoxe_sr)

print('Original Recording')
display(Audio(talgoxe, rate=talgoxe_sr))

print('High Pass Filtered Recording')
display(Audio(filtered_talgoxe, rate=talgoxe_sr))

## Testing the method for 2D signals
In this example we filter out low frequencies (High pass filtering) from a [Talgoxe](https://www.wikiwand.com/en/Great%20tit) (Great tit). As we can see in the image below, the view onto the bird seems to be blocked by some object, perhaps another tree's twigs. Through filtering out these larger areas of the same color we could get a better view of the actual structure of the bird. One could imagine this finds application in the detection and classification of wild animals. High pass filtering such an image would help a classification model to even out the playfield for all input images in a preprocessing step by getting rid of such "unwanted irregularities".

In [ ]:
talgoxe = cv2.imread(IMAGE_FOLDER+'talgoxe.jpg', cv2.IMREAD_GRAYSCALE)
filtered_talgoxe = filter_frequencies(talgoxe, (2, 10))

plot_fourier_spectrum(talgoxe, filtered_talgoxe)

_, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].imshow(talgoxe, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')
axes[1].imshow(filtered_talgoxe, cmap='gray')
axes[1].axis('off')
axes[1].set_title('Highpass Filtered Image')

By filtering out this small "ring" in the low frequency part of the Fourier Spectrum, we can suppress large areas of a color (in this example that would mean we suppress the large black bars that seem to block the view onto the bird).

# Task 2.3 - Filtering in the spatial and spectral domain
Wavelets are mathematical functions that can be used to decompose a signal into different frequency components, each with a different scale or resolution. The wavelet transform allows for the analysis of a signal at multiple resolutions by "sliding" and "scaling" the wavelet function over the signal, capturing **both frequency and time information**. Unlike the Fourier transform, which represents signals as a sum of sinusoids of various frequencies, the wavelet transform uses localized wavelets, providing a more flexible representation especially for signals with non-stationary or time-varying characteristics.

#### Example 1 - Wavelet Feature Extraction
In this experiment, we can imagine that we would like to give the road maintenance bureau of Malmö a tool to easily detect structural damages in the [Öresund Bridge](https://www.wikiwand.com/en/%C3%98resund_Bridge), which connects Denmark to Sweden. Since structural damages in the streets concrete usually identify themselves as narrow cracks, the goal here would be to filter high frequency intersections of the detail coefficients.

The following code performs a single-level 2D Discrete Wavelet Transform (DWT) using the "haar" wavelet. This decomposes the image into four components:

- `cA`: Approximation coefficients (low-frequency content).
- `cH`: Horizontal detail coefficients (horizontal features or edges).
- `cV`: Vertical detail coefficients (vertical features or edges).
- `cD`: Diagonal detail coefficients (diagonal features or edges).

We set the `cA` coefficients to `0` since we only want to focus on edges on the concrete's surface and therefore ignore the images color-surfaces.

A street's concrete is usually a rough surface, so it has a lot of small edges and bumps as it is made out of cement. We wouldn't want every little cement aggregate to show up on our filtered image so a step after filtering out the low-frequency components would be to threshold the edges and then show our result.

In [ ]:
import pywt

bridge = cv2.imread(IMAGE_FOLDER+"oresund_bridge.jpg", cv2.IMREAD_GRAYSCALE)

coeffs = pywt.dwt2(bridge, 'db1')
cA, (cH, cV, cD) = coeffs

cA.fill(0)

features = pywt.idwt2((cA, (cH, cV, cD)), 'db1')

features_normalized = cv2.normalize(features, None, 0, 255, cv2.NORM_MINMAX)

_, thresholded_features = cv2.threshold(features_normalized, 90, 255, cv2.THRESH_BINARY)

fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].imshow(bridge, cmap='gray')
ax[0].set_title('Original Image')
ax[0].axis('off')
ax[1].imshow(np.uint8(thresholded_features), cmap='gray')
ax[1].set_title('High Frequencies Thresholded')
ax[1].axis('off')

One might ask why the **Haar Wavelet** is suitable for this example: An argument for why I used the **Haar Wavelet** is because the street represents a large surface of more or less 2-3  graytones. A crack in the street usually identifies itself in a darker way as we can see in the original image's plot. Therefore a crack symbolizes a rapid change, which a Haar Wavelet is known to detect. Other wavelets also work but they can also include more suptle changes. 

**This is what the Haar Wavelet looks like:**

![Haar Wavelet](./images/notebook_images/Haar_wavelet.png)

To conclude this first usecase of wavelets we can definitely more easily structural differences in the image so a CNN would probably have  a much easier time segmenting the image and detecting such irregularities.

#### Example 2 - Wavelet Compression
Looking at the filtering in the spectral and spatial domain doesn't only yield for extraction purposes but it can also be used to compress images. A very famous image compression standard called [JPEG 2000](https://www.wikiwand.com/en/JPEG_2000) also uses wavelet transform in order to compress images.

In this experiment, or rather demonstration, we try to implement such a compression by dumping details of wavelet coefficients.

In [ ]:
ystad = cv2.imread(IMAGE_FOLDER+"ystad.jpg")

plt.imshow(cv2.cvtColor(ystad, cv2.COLOR_BGR2RGB))
plt.title('Image to compress')
plt.show()

The function below first gathers all coefficients of the image by switching into the 2D discrete wavelet space.

Then the actual compression happens; By "quantizing" the coefficients, we essentially **round** these memory-heavy numbers to numbers that require much less space. In this function we just quantize the coefficients to the nearest multiple of `quantize_level`. If these coefficients are smaller than our `threshold` we set them to `0` - These will multiply some of our wavelet components with `0` and therefore dumping them entirely.

**Note on the wavelet**: Unlike in the first demonstration/experiment we don't use the Haar-Wavelet here - instead we introduce the usage of the [Daubechies-Wavelet](https://www.wikiwand.com/en/Daubechies_wavelet) or `db1` in the `pywt` package. The Daubechies Wavelet has a few advantages when it comes to compression:
- The Daubechies wavelets are **orthogonal** to each other. This orthogonality ensures that the wavelet coefficients capture independent information about the signal, which is crucial for both signal reconstruction and compression.
- Daubechies wavelets are smooth and have good regularity properties. Smooth wavelets are often better at approximating smooth functions, which is beneficial for compression because natural signals and images often have smooth regions.
- Daubechies wavelets tend to concentrate energy into fewer, larger coefficients, which is a desirable property for compression. This is because large coefficients can be quantized more coarsely without significant perceptual loss in the reconstructed signal, while small coefficients can be discarded or quantized finely.

In [ ]:
def wavelet_compress(img, wavelet='db1', level=1, quantize_level=16, threshold=5):
    coeffs = pywt.wavedec2(img, wavelet=wavelet, level=level)
    
    original_coeff_count = coeffs[0].size + sum(detail[0].size + detail[1].size + detail[2].size for detail in coeffs[1:])
    compressed_coeffs = []

    approx_compressed = np.round(coeffs[0] / quantize_level) * quantize_level
    approx_compressed[np.abs(approx_compressed) < threshold] = 0
    compressed_coeffs.append(approx_compressed)
    
    compressed_coeff_count = np.count_nonzero(approx_compressed)

    for detail in coeffs[1:]:
        compressed_detail = (
            np.round(detail[0] / quantize_level) * quantize_level,
            np.round(detail[1] / quantize_level) * quantize_level,
            np.round(detail[2] / quantize_level) * quantize_level
        )
        
        for d in compressed_detail:
            d[np.abs(d) < threshold] = 0

        compressed_coeffs.append(compressed_detail)
        
        compressed_coeff_count += sum(np.count_nonzero(c) for c in compressed_detail)

    reduction_percent = 100 * (1 - (compressed_coeff_count / original_coeff_count))

    print(f"Original number of Coefficients: {original_coeff_count}")
    print(f"Compressed number of Coefficients: {compressed_coeff_count}")
    print(f"Reduction in Coefficients: {reduction_percent:.2f}%")
    
    return compressed_coeffs

To decompress our image (or rather reproduce the original) we can just call the `pywt.waverec2` function and pass it the much smaller list of coefficients, it then clips the reconstructed data to the conventional `0` to `255` range.

In [ ]:
def wavelet_decompress(compressed_coeffs, wavelet='db1'):
    waverec = pywt.waverec2(compressed_coeffs, wavelet=wavelet)
    img_reconstructed = np.clip(waverec, 0, 255).astype(np.uint8)
    
    return img_reconstructed

This function handles the entire demonstration process of the wavelet-compression.

In [ ]:
def process_compression(img, wavelet='db1'):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Original Image')
    plt.show()
    
    print(f'Image resolution: {img.shape[0]} x {img.shape[1]} pixels')

    compressed_coeffs = wavelet_compress(img, wavelet=wavelet)
    img_reconstructed = wavelet_decompress(compressed_coeffs, wavelet=wavelet)

    plt.imshow(cv2.cvtColor(img_reconstructed, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Reconstructed Image')
    plt.show()

In [ ]:
process_compression(ystad)

It is important to note that although this method "compresses" the image, it does not necessarily reduce disk space unless we would implement another method of storing the compressed image that efficiently encodes the zero coefficients (e.g. run-length encoding or Huffman encoding). The above example is intended to demonstrate the compression process, not necessarily to achieve file size reduction.

The metrics we chose to display though, show that we were able to reduce the **number of coefficients** (and therefore components) by about `72%` which is quite a lot. The reconstructed image shows almost no difference so our algorithm did its job.

# Task 2.4 - Algorithms to detect structures in images

We have reached the last step in our journey through Sweden! We are now in the southernmost city of Sweden - Välkommen till Malmö.

Malmö's most iconic structure is the [Turning Torso](https://www.wikiwand.com/en/Turning_Torso), a neo-futurist skyscraper with a height of 192m, which is quite high for European standards. In this last task we are going to try to construct a corner detector that can detect the building's complex corners.

#### What are corners even?

When looking at corners in images; A corner refers to a location where its immediate surroundings exhibit two prevalent and distinct edge orientations. Essentially, a corner can be understood as the convergence point of two edges, with an edge being defined as a noticeable shift in image luminosity.
![Image explaining corners](images/notebook_images/corners.webp)

### Making use of the Harris Corner Detector
The [Harris Corner Detector](https://www.wikiwand.com/en/Harris_corner_detector) is a common algorithm to detect corners in images.

#### How the algorithm works
1. Convert to grayscale color space
2. Apply Sobel operator to find x- and y-gradients of every pixel
3. Compute the Structure Tensor
3. With an $n\times n$-Window around every pixel, compute the **Harris Value** (using a strength function - This is usually SSD)
4. Define and find a threshold of pixels that imply a local maxima of a given local window (to reduce redundant corner detection)
5. For each pixel in *Step 5*, compute a feature descriptor

### Step 1: Converting the Image to Greyscale

In [ ]:
torso = cv2.imread(IMAGE_FOLDER+'torso.jpg', cv2.IMREAD_COLOR)
torso = cv2.cvtColor(torso, cv2.COLOR_BGR2RGB)
torso_g = cv2.cvtColor(torso, cv2.COLOR_RGB2GRAY)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(torso)
axes[0].axis('off')
axes[0].set_title('RGB Image')
axes[1].imshow(torso_g, cmap='gray')
axes[1].axis('off')
axes[1].set_title('Grayscale Image')
fig.suptitle('Images of the Turning Torso')

### Step 2: Apply a Sobel Filter
In the seconds step we apply a [Sobel Filter](https://www.wikiwand.com/en/Sobel_operator) in each cartesian direction (on the x- and y-axis). Through this step we get the derivatives along both axes - Which we will need further on for the Gaussian window.

In [ ]:
# apply sobel operator
sobel_x = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
])
sobel_y = np.array([
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]
])

torso_g_x = apply_2d_filter(torso_g, sobel_x)
torso_g_y = apply_2d_filter(torso_g, sobel_y)

In [ ]:
# plot sobel x and y
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].imshow(torso_g_x, cmap='gray')
axes[0].axis('off')
axes[0].set_title('Sobel X')
axes[1].imshow(torso_g_y, cmap='gray')
axes[1].axis('off')
axes[1].set_title('Sobel Y')
fig.suptitle('Sobel X and Y')

Both the x-axis and y-axis Sobel filter give us the images edges along each axis as shown in the plots.

### Step 3: Calculating the components of the Structure Tensor
For each pixel, calculate the products of derivatives, $I_x^2$, $I_y^2$ and $I_x*I_y$ over all pixels in a window.

In [ ]:
I_xx = np.square(torso_g_x)
I_yy = np.square(torso_g_y)
I_xy = torso_g_x * torso_g_y

### Step 4: Harris Response Calculation
For each pixel, calculate the Harris response R using determinant and trace of the matrix created by sums of the products calculated in step 2, typically using the following formula: 
$$ R = det(M)-k(trace(M))^2$$ 
Where 
- $M$ is the matrix of sums of the products of derivatives
- $k$ is a free sensitivity factor, usually chosen empirically
- $det(M) = \lambda_1*\lambda_2$
- $trace(M) = \lambda_1 + \lambda_2$
- $\lambda_1$ and $\lambda_2$ are the **Eigenvalues** of $M$

In [ ]:
k = 0.04
threshold = 0.3

Sxx = cv2.GaussianBlur(I_xx, (5,5), 1)
Syy = cv2.GaussianBlur(I_yy, (5,5), 1)
Sxy = cv2.GaussianBlur(I_xy, (5,5), 1)
    
detM = Sxx * Syy - Sxy**2
traceM = Sxx + Syy
R = detM - k * (traceM**2)

### Step 5: Threshold the computed corners

In [ ]:
corner_image = np.zeros_like(R, dtype=np.float32)
corner_image[R > threshold * R.max()] = 255

plt.imshow(corner_image, cmap='gray')

### Step 6: Compute a feature descriptor and show the result

In [ ]:
torso = cv2.cvtColor(torso_g, cv2.COLOR_GRAY2BGR)

for y in range(corner_image.shape[0]):
    for x in range(corner_image.shape[1]):
        if corner_image[y, x] == 255:
            cv2.circle(torso, (x, y), 3, (0, 0, 255), 1, 2)

plt.imshow(cv2.cvtColor(torso, cv2.COLOR_BGR2RGB))
plt.show()

As we can see, our corner detector seems to work. It doesn't detect every single corner but most of them. The difficult part are the small edges which have a tiny bit of blur. Corners with great contrast differences seem to be detected the best. 

To validate the detector, let's look at another sample, a set of [Kubb](https://www.wikiwand.com/en/Kubb), the viking's take on chess.

In [ ]:
# Step 1: Convert the image to grayscale
kubb = cv2.imread(IMAGE_FOLDER+'kubb.png', cv2.IMREAD_COLOR)
kubb = cv2.cvtColor(kubb, cv2.COLOR_BGR2RGB)
kubb_g = cv2.cvtColor(kubb, cv2.COLOR_RGB2GRAY)

# Step 2: Apply the Sobel operator to the image
kubb_g_x = apply_2d_filter(kubb_g, sobel_x)
kubb_g_y = apply_2d_filter(kubb_g, sobel_y)

# Step 3: Calculate the structure tensor
I_xx = np.square(kubb_g_x)
I_yy = np.square(kubb_g_y)
I_xy = kubb_g_x * kubb_g_y

# Step 4: Calculating the Harris Response
k = 0.005
threshold = 0.09

Sxx = cv2.GaussianBlur(I_xx, (5,5), 1)
Syy = cv2.GaussianBlur(I_yy, (5,5), 1)
Sxy = cv2.GaussianBlur(I_xy, (5,5), 1)
    
detM = Sxx * Syy - Sxy**2
traceM = Sxx + Syy
R = detM - k * (traceM**2)

# Step 5: Thresholding the Harris Response
corner_image = np.zeros_like(R, dtype=np.float32)
corner_image[R > threshold * R.max()] = 255

# Step 6:
kubb = cv2.cvtColor(kubb_g, cv2.COLOR_GRAY2BGR)

for y in range(corner_image.shape[0]):
    for x in range(corner_image.shape[1]):
        if corner_image[y, x] == 255:
            cv2.circle(kubb, (x, y), 3, (0, 0, 255), 1, 2)

plt.imshow(cv2.cvtColor(kubb, cv2.COLOR_BGR2RGB))
plt.show()

The same observation can be made here. Corners in the shadow, next to the white background get detected easily - Corners which are at bright spots of the wood have a hard time being detected.

An idea to improve this would be to take further preprocessing steps between step 1 and step 2, perhaps a binarization of the image could help.